In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/model-dataset-new-1/2015.csv
/kaggle/input/model-dataset-new-1/2017.csv
/kaggle/input/model-dataset-new-1/2019.csv
/kaggle/input/model-dataset-new-1/main_data.csv
/kaggle/input/model-dataset-new-1/2018.csv
/kaggle/input/model-dataset-new-1/2016.csv


In [20]:
!pip install gtts

In [1]:

import pandas as pd
import logging
from sklearn.preprocessing import StandardScaler
from transformers import BartTokenizer, BartForConditionalGeneration
import logging
import re
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from PIL import Image
import random
from moviepy.editor import ImageSequenceClip, AudioFileClip, concatenate_videoclips
import os
import time
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import os
from PIL import Image
from moviepy.editor import ImageSequenceClip, AudioFileClip, concatenate_videoclips, CompositeVideoClip, TextClip
import numpy as np
import io
import pandas as pd
import time
from matplotlib.animation import FuncAnimation
from gtts import gTTS
from transformers import BartTokenizer, BartForConditionalGeneration
import spacy
import re
import pandas as pd
import logging
from sklearn.preprocessing import StandardScaler
import pandas as pd
import logging
from sklearn.preprocessing import StandardScaler
import pandas as pd
import logging
from sklearn.preprocessing import StandardScaler
import pickle
from transformers import BartTokenizer, BartForConditionalGeneration
import spacy
import os
import logging
import warnings 
import os
import joblib
from transformers import BartTokenizer, BartForConditionalGeneration
import spacy
import logging


# nlp = spacy.load("en_core_web_sm")

# Setting up logging
logging.basicConfig(level=logging.INFO)

# # Load spaCy model for NER
# nlp = spacy.load("en_core_web_sm")


c:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
c:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\transforms\v2\__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Pl

In [2]:
# load and Preprocess the Data 

In [3]:
def load_and_preprocess_data(file_path):
    try:
        logging.info(f"Loading data from {file_path}")
        # Load data
        if file_path.endswith('.csv'):
            data = pd.read_csv(file_path)
        elif file_path.endswith('.xlsx'):
            data = pd.read_excel(file_path)
        elif file_path.endswith('.txt'):
            data = pd.read_csv(file_path, delimiter='\t')
        else:
            raise ValueError("Unsupported file format.")
        
        logging.info("Data loaded successfully")
        
        # Detect and convert data types
        for col in data.columns:
            try:
                data[col] = pd.to_numeric(data[col], errors='coerce')
            except ValueError:
                pass
        
        # Handle missing values
        data.fillna(data.mean(), inplace=True)
        
        # Handle categorical data
        categorical_cols = data.select_dtypes(include=['object']).columns
        data = pd.get_dummies(data, columns=categorical_cols, drop_first=True)
        
        # Normalize numeric data
        numeric_cols = data.select_dtypes(include=['float64', 'int64']).columns
        scaler = StandardScaler()
        data[numeric_cols] = scaler.fit_transform(data[numeric_cols])
        
        logging.info("Data preprocessing completed")
        return data
    except Exception as e:
        logging.error(f"Error loading and preprocessing data: {e}")
        raise

In [4]:
import logging
import pandas as pd

def perform_eda(data):
    try:
        logging.info("Performing EDA...")
        
        # Basic EDA summary
        eda_summary = {
            "shape": data.shape,
            "columns": data.columns.tolist(),
            "dtypes": data.dtypes.apply(lambda x: x.name).to_dict(),
            "null_counts": data.isnull().sum().to_dict(),
            "describe": data.describe(include='all').to_dict()
        }
        
        # Correlation analysis
        numeric_cols = data.select_dtypes(include=['float64', 'int64']).columns
        if len(numeric_cols) > 1:
            eda_summary["correlation"] = data[numeric_cols].corr().to_dict()
        
        # Unique value counts
        eda_summary["unique_counts"] = {col: data[col].nunique() for col in data.columns}
        
        # Missing value analysis
        eda_summary["missing_value_analysis"] = {
            "total_missing": data.isnull().sum().sum(),
            "missing_per_column": data.isnull().sum().to_dict(),
            "missing_percentage_per_column": (data.isnull().sum() / len(data) * 100).to_dict()
        }
        
        # Categorical data analysis
        categorical_cols = data.select_dtypes(include=['object']).columns
        eda_summary["categorical_summary"] = {
            col: data[col].value_counts().to_dict() for col in categorical_cols
        }
        
        # Outlier detection
        eda_summary["outliers"] = {
            col: data[col][((data[col] - data[col].mean()) / data[col].std()).abs() > 3].tolist() for col in numeric_cols
        }
        
        # Distribution analysis
        eda_summary["distribution"] = {
            col: data[col].describe().to_dict() for col in numeric_cols
        }
        
        # Time series analysis
        date_cols = [col for col in data.columns if 'date' in col.lower() or 'time' in col.lower()]
        if date_cols:
            eda_summary["time_series_analysis"] = {
                col: data.set_index(col).resample('M').mean().to_dict() for col in date_cols if pd.api.types.is_datetime64_any_dtype(data[col])
            }
        
        logging.debug(f"EDA Summary: {eda_summary}")
        return eda_summary
    except Exception as e:
        logging.error(f"Error performing EDA: {e}")
        raise

In [185]:
def save_models(model_name="facebook/bart-large-cnn", save_directory="models"):
    if not os.path.exists(save_directory):
        os.makedirs(save_directory)
    
    # Load the tokenizer and model
    tokenizer = BartTokenizer.from_pretrained(model_name)
    model = BartForConditionalGeneration.from_pretrained(model_name)
    
    # Save the tokenizer and model as .pkl files
    with open(os.path.join(save_directory, "facebook_tokenizer.pkl"), "wb") as f:
        pickle.dump(tokenizer, f)
    with open(os.path.join(save_directory, "facebook_model.pkl"), "wb") as f:
        pickle.dump(model, f)
    
    # Load and save the spaCy model
    nlp = spacy.load("en_core_web_sm")
    with open(os.path.join(save_directory, "spacy_model.pkl"), "wb") as f:
        pickle.dump(nlp, f)
    
    logging.info(f"Models saved to {save_directory}")

# Call this function once to save the models
save_models()

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [5]:
def load_models(model_directory="models"):
    # Load the tokenizer and model from the saved .pkl files
    with open(os.path.join(model_directory, "facebook_tokenizer.pkl"), "rb") as f:
        tokenizer = pickle.load(f)
    with open(os.path.join(model_directory, "facebook_model.pkl"), "rb") as f:
        model = pickle.load(f)
    
    # Load the spaCy model from the saved .pkl file
    with open(os.path.join(model_directory, "spacy_model.pkl"), "rb") as f:
        nlp = pickle.load(f)
    
    return tokenizer, model, nlp

# Load the models
tokenizer, model, nlp = load_models()

In [6]:
import re
import logging
from transformers import BartTokenizer, BartForConditionalGeneration

def analyze_prompt_for_insights(prompt, eda_summary):
    try:
        logging.info("Analyzing prompt for insights")
        
        inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
        input_ids = inputs.input_ids
        attention_mask = inputs.attention_mask
        
        outputs = model.generate(
            input_ids,
            attention_mask=attention_mask,
            max_length=200,
            num_return_sequences=1,
            temperature=0.7,
            top_p=0.9,
            top_k=50,
            no_repeat_ngram_size=2,
            pad_token_id=tokenizer.eos_token_id
        )
        
        generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        logging.debug(f"Generated text from model: {generated_text}")
        
        insights, columns = extract_insights_and_columns_from_text(generated_text, eda_summary)
        
        if not insights:
            logging.warning("No insights could be extracted from the provided prompt. Using fallback insights.")
            insights = fallback_insights(eda_summary)
        
        if not columns:
            logging.warning("No columns could be extracted from the provided prompt. Using fallback columns.")
            columns = fallback_columns(eda_summary)
        
        logging.info(f"Insights extracted: {insights}")
        logging.info(f"Columns extracted: {columns}")
        return insights, columns
    except Exception as e:
        logging.error(f"Error in analyzing prompt for insights: {e}")
        return [], []

def extract_insights_and_columns_from_text(text, eda_summary):
    possible_insights = ["trend", "comparison", "distribution", "correlation", "pattern", "anomaly", "outlier", "relationship", "performance", "growth"]
    insights = []
    columns = []
    
    text = text.lower()
    
    for insight in possible_insights:
        if re.search(r'\b' + re.escape(insight) + r'\b', text):
            insights.append(insight)
    
    # Extract column names from the text
    # Assuming column names are mentioned in the text and are separated by commas
    column_pattern = re.compile(r'\b(?:columns|fields|attributes|features)\b\s*:\s*([\w\s,]+)')
    match = column_pattern.search(text)
    if match:
        columns = [col.strip() for col in match.group(1).split(',')]
    
    # Validate columns against EDA summary
    valid_columns = [col for col in columns if col in eda_summary['columns']]
    
    return insights, valid_columns

def fallback_insights(eda_summary):
    # Fallback logic to generate default insights based on EDA summary
    insights = []
    if len(eda_summary['columns']) > 1:
        insights.append("correlation")
    if any('date' in col.lower() or 'time' in col.lower() for col in eda_summary['columns']):
        insights.append("trend")
    if any(eda_summary['dtypes'][col] == 'object' for col in eda_summary['columns']):
        insights.append("distribution")
    return insights

def fallback_columns(eda_summary):
    # Fallback logic to generate default columns based on EDA summary
    numeric_columns = [col for col in eda_summary['columns'] if eda_summary['dtypes'][col] in ['float64', 'int64']]
    if len(numeric_columns) >= 2:
        return numeric_columns[:2]
    return numeric_columns
# def generate_frames(data, insights, columns):
#     if not data.empty and columns:
#         # Generate frames based on insights and columns
#         frames = []  # Placeholder for actual frame generation logic
#         logging.info(f"Generated {len(frames)} frames based on insights and columns.")
#         return frames
#     else:
#         logging.error("Value error: No frames generated from data.")
#         return []

# # Call the function
# insights, columns = analyze_prompt_for_insights(prompt, eda_summary)

# # Generate frames
# frames = generate_frames(data, insights, columns)

# # Print the results
# print(f"Insights: {insights}")
# print(f"Columns: {columns}")
# print(f"Frames: {frames}")


In [7]:
import logging
import io
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from matplotlib.animation import FuncAnimation
import imageio

def generate_animated_frames(data, insights, columns, output_dir="frames"):
    logging.info("Generating animated frames from data")
    frames = []
    
    # Ensure the output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Log the data columns and types
    logging.debug(f"Data columns: {data.columns}")
    logging.debug(f"Data types: {data.dtypes}")
    
    # Generate frames based on insights
    for insight in insights:
        logging.debug(f"Processing insight: {insight}")
        if insight == "trend":
            logging.info("Generating trend frames")
            date_column = None
            for col in data.columns:
                if 'date' in col.lower() or 'time' in col.lower():
                    date_column = col
                    break
            
            if date_column and columns:
                numeric_columns = [col for col in columns if col in data.columns and data[col].dtype in ['float64', 'int64']]
                if len(numeric_columns) > 0:
                    fig, ax = plt.subplots(figsize=(10, 6))
                    lines = [ax.plot([], [], label=col)[0] for col in numeric_columns]
                    ax.set_xlim(data[date_column].min(), data[date_column].max())
                    ax.set_ylim(data[numeric_columns].min().min(), data[numeric_columns].max().max())
                    ax.set_title(f"Time Series Plot for {', '.join(numeric_columns)}")
                    ax.set_xlabel('Time')
                    ax.set_ylabel('Values')
                    ax.legend()

                    def update(frame):
                        for line, col in zip(lines, numeric_columns):
                            line.set_data(data[date_column][:frame], data[col][:frame])
                        return lines

                    ani = FuncAnimation(fig, update, frames=len(data), blit=True)
                    for i in range(len(data)):
                        update(i)
                        buf = io.BytesIO()
                        fig.savefig(buf, format='png')
                        buf.seek(0)
                        img = Image.open(buf)
                        frames.append(np.array(img))
                    plt.close(fig)
                else:
                    logging.warning("No numeric columns found for trend insight")
            else:
                logging.warning("No date column or relevant columns found for trend insight")
        
        elif insight == "comparison":
            logging.info("Generating comparison frames")
            if len(columns) == 2 and all(col in data.columns for col in columns):
                fig, ax = plt.subplots(figsize=(10, 6))
                categories = data[columns[0]].unique()
                bars = ax.bar(categories, np.zeros(len(categories)))
                ax.set_ylim(0, data[columns[1]].max())
                ax.set_title('Animated Bar Chart')
                ax.set_xlabel(columns[0])
                ax.set_ylabel(columns[1])

                def update_bar(frame):
                    for bar, category in zip(bars, categories):
                        bar.set_height(data[data[columns[0]] == category][columns[1]].iloc[frame])
                    return bars

                ani = FuncAnimation(fig, update_bar, frames=len(data), blit=True)
                for i in range(len(data)):
                    update_bar(i)
                    buf = io.BytesIO()
                    fig.savefig(buf, format='png')
                    buf.seek(0)
                    img = Image.open(buf)
                    frames.append(np.array(img))
                plt.close(fig)
            else:
                logging.warning("Required columns for comparison insight not found or incorrect")
        
        elif insight == "distribution":
            logging.info("Generating distribution frames")
            numeric_columns = [col for col in columns if col in data.columns and data[col].dtype in ['float64', 'int64']]
            if len(numeric_columns) > 0:
                for col in numeric_columns:
                    fig, ax = plt.subplots(figsize=(8, 6))
                    sns.histplot(data[col], bins=10, kde=True, ax=ax)
                    ax.set_title(f"Distribution of {col}")
                    buf = io.BytesIO()
                    fig.savefig(buf, format='png')
                    buf.seek(0)
                    img = Image.open(buf)
                    frames.append(np.array(img))
                    plt.close(fig)
            else:
                logging.warning("No numeric columns found for distribution insight")
        
        elif insight == "correlation":
            logging.info("Generating correlation frames")
            numeric_columns = [col for col in columns if col in data.columns and data[col].dtype in ['float64', 'int64']]
            if len(numeric_columns) > 1:
                corr_matrix = data[numeric_columns].corr()
                fig, ax = plt.subplots(figsize=(10, 6))
                sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5, ax=ax)
                ax.set_title('Correlation Matrix')
                buf = io.BytesIO()
                fig.savefig(buf, format='png')
                buf.seek(0)
                img = Image.open(buf)
                frames.append(np.array(img))
                plt.close(fig)
            else:
                logging.warning("Not enough numeric columns for correlation insight")
        
        elif insight == "pie":
            logging.info("Generating pie chart frames")
            if len(columns) == 2 and all(col in data.columns for col in columns):
                fig, ax = plt.subplots(figsize=(10, 6))
                def update_pie(frame):
                    ax.clear()
                    ax.pie(data[columns[1]][:frame], labels=data[columns[0]][:frame], autopct='%1.1f%%')
                    ax.set_title('Animated Pie Chart')

                ani = FuncAnimation(fig, update_pie, frames=len(data), blit=True)
                for i in range(len(data)):
                    update_pie(i)
                    buf = io.BytesIO()
                    fig.savefig(buf, format='png')
                    buf.seek(0)
                    img = Image.open(buf)
                    frames.append(np.array(img))
                plt.close(fig)
            else:
                logging.warning("Required columns for pie chart insight not found or incorrect")
        
        elif insight == "pattern":
            logging.info("Generating pattern frames")
            # Example: Detecting and animating a repeating pattern in a time series
            date_column = None
            for col in data.columns:
                if 'date' in col.lower() or 'time' in col.lower():
                    date_column = col
                    break
            
            if date_column and columns:
                numeric_columns = [col for col in columns if col in data.columns and data[col].dtype in ['float64', 'int64']]
                if len(numeric_columns) > 0:
                    fig, ax = plt.subplots(figsize=(10, 6))
                    lines = [ax.plot([], [], label=col)[0] for col in numeric_columns]
                    ax.set_xlim(data[date_column].min(), data[date_column].max())
                    ax.set_ylim(data[numeric_columns].min().min(), data[numeric_columns].max().max())
                    ax.set_title(f"Pattern Detection for {', '.join(numeric_columns)}")
                    ax.set_xlabel('Time')
                    ax.set_ylabel('Values')
                    ax.legend()

                    def update(frame):
                        for line, col in zip(lines, numeric_columns):
                            line.set_data(data[date_column][:frame], data[col][:frame])
                        return lines

                    ani = FuncAnimation(fig, update, frames=len(data), blit=True)
                    for i in range(len(data)):
                        update(i)
                        buf = io.BytesIO()
                        fig.savefig(buf, format='png')
                        buf.seek(0)
                        img = Image.open(buf)
                        frames.append(np.array(img))
                    plt.close(fig)
                else:
                    logging.warning("No numeric columns found for pattern insight")
            else:
                logging.warning("No date column or relevant columns found for pattern insight")

        elif insight == "anomaly":
            logging.info("Generating anomaly frames")
            # Example: Highlighting anomalies in a time series
            date_column = None
            for col in data.columns:
                if 'date' in col.lower() or 'time' in col.lower():
                    date_column = col
                    break
            
            if date_column and columns:
                numeric_columns = [col for col in columns if col in data.columns and data[col].dtype in ['float64', 'int64']]
                if len(numeric_columns) > 0:
                    fig, ax = plt.subplots(figsize=(10, 6))
                    lines = [ax.plot([], [], label=col)[0] for col in numeric_columns]
                    ax.set_xlim(data[date_column].min(), data[date_column].max())
                    ax.set_ylim(data[numeric_columns].min().min(), data[numeric_columns].max().max())
                    ax.set_title(f"Anomaly Detection for {', '.join(numeric_columns)}")
                    ax.set_xlabel('Time')
                    ax.set_ylabel('Values')
                    ax.legend()

                    def update(frame):
                        for line, col in zip(lines, numeric_columns):
                            line.set_data(data[date_column][:frame], data[col][:frame])
                            if data[col][frame] > data[col].mean() + 3 * data[col].std() or data[col][frame] < data[col].mean() - 3 * data[col].std():
                                line.set_color('red')
                            else:
                                line.set_color('blue')
                        return lines

                    ani = FuncAnimation(fig, update, frames=len(data), blit=True)
                    for i in range(len(data)):
                        update(i)
                        buf = io.BytesIO()
                        fig.savefig(buf, format='png')
                        buf.seek(0)
                        img = Image.open(buf)
                        frames.append(np.array(img))
                    plt.close(fig)
                else:
                    logging.warning("No numeric columns found for anomaly insight")
            else:
                logging.warning("No date column or relevant columns found for anomaly insight")

        elif insight == "outlier":
            logging.info("Generating outlier frames")
            # Example: Highlighting outliers in a scatter plot
            if len(columns) == 2 and all(col in data.columns for col in columns):
                fig, ax = plt.subplots(figsize=(10, 6))
                scatter = ax.scatter(data[columns[0]], data[columns[1]], c='blue')
                ax.set_title('Outlier Detection')
                ax.set_xlabel(columns[0])
                ax.set_ylabel(columns[1])

                def update(frame):
                    ax.clear()
                    ax.scatter(data[columns[0]], data[columns[1]], c='blue')
                    outliers = data[(data[columns[1]] > data[columns[1]].mean() + 3 * data[columns[1]].std()) | (data[columns[1]] < data[columns[1]].mean() - 3 * data[columns[1]].std())]
                    ax.scatter(outliers[columns[0]], outliers[columns[1]], c='red')
                    ax.set_title('Outlier Detection')
                    ax.set_xlabel(columns[0])
                    ax.set_ylabel(columns[1])
                    return scatter,

                ani = FuncAnimation(fig, update, frames=len(data), blit=True)
                for i in range(len(data)):
                    update(i)
                    buf = io.BytesIO()
                    fig.savefig(buf, format='png')
                    buf.seek(0)
                    img = Image.open(buf)
                    frames.append(np.array(img))
                plt.close(fig)
            else:
                logging.warning("Required columns for outlier insight not found or incorrect")

        elif insight == "relationship":
            logging.info("Generating relationship frames")
            # Example: Showing relationship between two variables over time
            if len(columns) == 2 and all(col in data.columns for col in columns):
                fig, ax = plt.subplots(figsize=(10, 6))
                scatter = ax.scatter(data[columns[0]], data[columns[1]], c='blue')
                ax.set_title('Relationship Over Time')
                ax.set_xlabel(columns[0])
                ax.set_ylabel(columns[1])

                def update(frame):
                    ax.clear()
                    ax.scatter(data[columns[0]][:frame], data[columns[1]][:frame], c='blue')
                    ax.set_title('Relationship Over Time')
                    ax.set_xlabel(columns[0])
                    ax.set_ylabel(columns[1])
                    return scatter,

                ani = FuncAnimation(fig, update, frames=len(data), blit=True)
                for i in range(len(data)):
                    update(i)
                    buf = io.BytesIO()
                    fig.savefig(buf, format='png')
                    buf.seek(0)
                    img = Image.open(buf)
                    frames.append(np.array(img))
                plt.close(fig)
            else:
                logging.warning("Required columns for relationship insight not found or incorrect")

        elif insight == "performance":
            logging.info("Generating performance frames")
            # Example: Showing performance metrics over time
            date_column = None
            for col in data.columns:
                if 'date' in col.lower() or 'time' in col.lower():
                    date_column = col
                    break
            
            if date_column and columns:
                numeric_columns = [col for col in columns if col in data.columns and data[col].dtype in ['float64', 'int64']]
                if len(numeric_columns) > 0:
                    fig, ax = plt.subplots(figsize=(10, 6))
                    lines = [ax.plot([], [], label=col)[0] for col in numeric_columns]
                    ax.set_xlim(data[date_column].min(), data[date_column].max())
                    ax.set_ylim(data[numeric_columns].min().min(), data[numeric_columns].max().max())
                    ax.set_title(f"Performance Metrics for {', '.join(numeric_columns)}")
                    ax.set_xlabel('Time')
                    ax.set_ylabel('Values')
                    ax.legend()

                    def update(frame):
                        for line, col in zip(lines, numeric_columns):
                            line.set_data(data[date_column][:frame], data[col][:frame])
                        return lines

                    ani = FuncAnimation(fig, update, frames=len(data), blit=True)
                    for i in range(len(data)):
                        update(i)
                        buf = io.BytesIO()
                        fig.savefig(buf, format='png')
                        buf.seek(0)
                        img = Image.open(buf)
                        frames.append(np.array(img))
                    plt.close(fig)
                else:
                    logging.warning("No numeric columns found for performance insight")
            else:
                logging.warning("No date column or relevant columns found for performance insight")

        elif insight == "growth":
            logging.info("Generating growth frames")
            # Example: Showing growth over time
            date_column = None
            for col in data.columns:
                if 'date' in col.lower() or 'time' in col.lower():
                    date_column = col
                    break
            
            if date_column and columns:
                numeric_columns = [col for col in columns if col in data.columns and data[col].dtype in ['float64', 'int64']]
                if len(numeric_columns) > 0:
                    fig, ax = plt.subplots(figsize=(10, 6))
                    lines = [ax.plot([], [], label=col)[0] for col in numeric_columns]
                    ax.set_xlim(data[date_column].min(), data[date_column].max())
                    ax.set_ylim(data[numeric_columns].min().min(), data[numeric_columns].max().max())
                    ax.set_title(f"Growth Over Time for {', '.join(numeric_columns)}")
                    ax.set_xlabel('Time')
                    ax.set_ylabel('Values')
                    ax.legend()

                    def update(frame):
                        for line, col in zip(lines, numeric_columns):
                            line.set_data(data[date_column][:frame], data[col][:frame])
                        return lines

                    ani = FuncAnimation(fig, update, frames=len(data), blit=True)
                    for i in range(len(data)):
                        update(i)
                        buf = io.BytesIO()
                        fig.savefig(buf, format='png')
                        buf.seek(0)
                        img = Image.open(buf)
                        frames.append(np.array(img))
                    plt.close(fig)
                else:
                    logging.warning("No numeric columns found for growth insight")
            else:
                logging.warning("No date column or relevant columns found for growth insight")
        
        else:
            logging.warning(f"Unhandled insight type: {insight}")
    
    logging.info(f"Generated {len(frames)} animated frames")
    
    return frames


In [8]:

def create_video_from_frames(frames, audio_file=None, video_file="final_production_model.mp4"):
    if not frames:
        raise ValueError("No frames to create video from.")
    
    logging.info("Creating video from frames")
    video_clips = []
    for frame in frames:
        img_clip = ImageSequenceClip([frame], fps=1)  # 1 frame per second
        img_clip = img_clip.set_duration(2)  # Each frame lasts 2 seconds
        video_clips.append(img_clip)
    
    video = concatenate_videoclips(video_clips, method="compose")
    
    if audio_file and os.path.isfile(audio_file):
        audio = AudioFileClip(audio_file)
        video = video.set_audio(audio)
    
    video.write_videofile(video_file, codec="libx264", fps=24)
    logging.info(f"Video saved as {video_file}")

def generate_infographic_video(data, insights, columns,audio_file=None, title_image="title_screen.png"):
    frames = generate_animated_frames(data, insights, columns)
    if not frames:
        raise ValueError("No frames generated from data.")
    
    if os.path.exists(title_image):
        title_image_clip = Image.open(title_image)
        title_image_clip = title_image_clip.convert("RGBA")
        title_image_clip = np.array(title_image_clip)
        frames.insert(0, title_image_clip)
    
    create_video_from_frames(frames, audio_file)
    print("Video successfully generated!")

In [9]:
def generate_narration(text, output_file="narration.mp3", lang='en', slow=False, tld='com'):
    """
    Generate narration audio from text using gTTS (Google Text-to-Speech).

    Parameters:
    text (str): The text to be converted to speech.
    output_file (str): The file path where the audio file will be saved.
    lang (str): The language in which the text will be spoken. Default is 'en' (English).
    slow (bool): Whether to speak slowly. Default is False.
    tld (str): Top-level domain for the Google Translate host. Default is 'com'.

    Returns:
    str: The file path of the saved audio file.
    """
    try:
        logging.info(f"Generating narration for text: {text}")
        tts = gTTS(text=text, lang=lang, slow=slow, tld=tld)
        tts.save(output_file)
        logging.info(f"Narration saved to {output_file}")
        return output_file
    except Exception as e:
        logging.error(f"Error generating narration: {e}")
        raise

# Example usage:
# narration_text = "This is a detailed narration of the data analysis results."
# narration_file = generate_narration(narration_text)
# print(f"Narration file saved at: {narration_file}")

In [10]:
def data_storytelling_pipeline(file_path, prompt):
    try:
        start_time = time.time()
        
        logging.info("Loading and preprocessing data...")
        data = load_and_preprocess_data(file_path)
        logging.debug(f"Loaded Data: {data.head()}")
        
        logging.info("Performing EDA...")
        eda_summary = perform_eda(data)
        logging.debug(f"EDA Summary: {eda_summary}")
        
        logging.info("Analyzing the user's prompt...")
        insights, columns = analyze_prompt_for_insights(prompt, eda_summary)
        logging.debug(f"Extracted insights: {insights}")
        logging.debug(f"Extracted columns: {columns}")
        
        logging.info("Generating narration...")
        narration_text = f"Here is the analysis based on the prompt: {prompt}. Insights: {', '.join(insights)}"
        narration_file = generate_narration(narration_text)
        
        logging.info("Creating the infographic video...")
        video_file = generate_infographic_video(data, insights, columns, audio_file=narration_file)
        
        end_time = time.time()
        logging.info(f"Pipeline completed successfully in {end_time - start_time:.2f} seconds")
        
        
        return video_file
    
    except FileNotFoundError as fnf_error:
        logging.error(f"File not found: {fnf_error}")
        raise
    except pd.errors.ParserError as parser_error:
        logging.error(f"Error parsing the file: {parser_error}")
        raise
    except TypeError as type_error:
        logging.error(f"Type error: {type_error}")
        raise
    except ValueError as value_error:
        logging.error(f"Value error: {value_error}")
        raise
    except Exception as e:
        logging.error(f"An unexpected error occurred: {e}")
        raise

In [11]:
# Example usage:
file_path = 'D:\\1OOx-enginners-hackathon-submission-2\\data\\2015.csv'
prompt = "analyse the region columns."
# audio_file = "D:\\1OOx-enginners-hackathon-submission-2\\uploads\\audio_files"
# title_image = "D:\\1OOx-enginners-hackathon-submission-2\\outputs"

if not os.path.exists(file_path):
    raise FileNotFoundError(f"Data file not found: {file_path}")
# if not os.path.exists(audio_file):
#     raise FileNotFoundError(f"Audio file not found: {audio_file}")
# if not os.path.exists(title_image):
#     raise FileNotFoundError(f"Title image not found: {title_image}")

video_file = data_storytelling_pipeline(file_path, prompt)
print(video_file)
# return video_file

INFO:root:Loading and preprocessing data...
INFO:root:Loading data from D:\1OOx-enginners-hackathon-submission-2\data\2015.csv
INFO:root:Data loaded successfully
  updated_mean = (last_sum + new_sum) / updated_sample_count

  T = new_sum / new_sample_count

  new_unnormalized_variance -= correction**2 / new_sample_count

INFO:root:Data preprocessing completed
INFO:root:Performing EDA...
INFO:root:Performing EDA...
INFO:root:Analyzing the user's prompt...
INFO:root:Analyzing prompt for insights
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
  warnings.warn(

  warnings.warn(

INFO:root:Insights extracted: ['correlation']
INFO:root:Columns extracted: ['Country', 'Region']
INFO:root:Generating narration...
INFO:root:Generating narration for text: Here is the analysis based on the prompt: analyse the region columns.. Insights: correlation
INFO:root:Narration saved to narration.mp3
INFO:root:Creati

Moviepy - Building video final_production_model.mp4.
MoviePy - Writing audio in final_production_modelTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video final_production_model.mp4



INFO:root:Video saved as final_production_model.mp4         
INFO:root:Pipeline completed successfully in 29.08 seconds


Moviepy - Done !
Moviepy - video ready final_production_model.mp4
Video successfully generated!
None


#### DONE ✅

# OLD MODEL CODE IMPLEMENTATION

In [20]:
# import plotly.express as px
# import pandas as pd
# import logging

# def generate_visualizations(data, insights):
#     try:
#         logging.info("Generating visualizations")
#         if data.empty:
#             raise ValueError("Data is empty.")
#         if not insights:
#             raise ValueError("No insights available to generate visualizations.")
        
#         visuals = []
#         numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns
#         categorical_columns = data.select_dtypes(include=['object', 'category']).columns
        
#         if 'Date' in data.columns or 'Datetime' in data.columns:
#             time_column = 'Date' if 'Date' in data.columns else 'Datetime'
#             if numeric_columns.any():
#                 fig = px.line(data, x=time_column, y=numeric_columns, title=f"Time Series Plot for {', '.join(numeric_columns)}")
#                 visuals.append(fig)
        
#         if len(numeric_columns) > 1:
#             corr_matrix = data[numeric_columns].corr()
#             fig = px.imshow(corr_matrix, text_auto=True, title='Correlation Matrix')
#             visuals.append(fig)
        
#         for col in categorical_columns:
#             fig = px.histogram(data, x=col, title=f"Distribution of {col}")
#             visuals.append(fig)
        
#         if len(numeric_columns) > 1:
#             fig = px.scatter_matrix(data, dimensions=numeric_columns, title='Pairwise Relationships')
#             visuals.append(fig)
        
#         if not visuals:
#             return None
        
#         logging.info(f"Generated {len(visuals)} visualizations")
#         return visuals
#     except Exception as e:
#         logging.error(f"Error generating visualizations: {e}")
#         raise

# # # Example usage:
# # file_path = '/kaggle/input/model-dataset-new-1/2015.csv'
# # data = pd.read_csv(file_path)
# # insights = "Sample insights based on the data"
# # visuals = generate_visualizations(data, insights)

# # # To display the visualizations in a Jupyter notebook
# # for fig in visuals:
# #     fig.show()

In [21]:
# import matplotlib.pyplot as plt
# import logging

# def generate_default_frames(data):
#     logging.info("Generating default frames from data")
#     frames = []
    
#     # Generate a basic time series plot if a date column is present
#     date_column = None
#     for col in data.columns:
#         if 'date' in col.lower() or 'time' in col.lower():
#             date_column = col
#             break
    
#     if date_column:
#         numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns
#         if len(numeric_columns) > 0:
#             plt.figure(figsize=(10, 6))
#             for col in numeric_columns:
#                 plt.plot(data[date_column], data[col], label=col)
#             plt.title(f"Time Series Plot for {', '.join(numeric_columns)}")
#             plt.xlabel('Time')
#             plt.ylabel('Values')
#             plt.xticks(rotation=45)
#             plt.legend()
#             frames.append(plt)
    
#     # Generate a correlation matrix plot
#     if data.select_dtypes(include=['float64', 'int64']).shape[1] > 1:
#         corr_matrix = data.corr()
#         plt.figure(figsize=(10, 6))
#         sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
#         plt.title('Correlation Matrix')
#         frames.append(plt)
    
#     # Generate distribution plots for numeric columns
#     numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns
#     for col in numeric_columns:
#         plt.figure(figsize=(8, 6))
#         sns.histplot(data[col], bins=10, kde=True)
#         plt.title(f"Distribution of {col}")
#         frames.append(plt)
    
#     logging.info(f"Generated {len(frames)} default frames")
#     return frames

In [22]:
# print(frames)

In [23]:
# from moviepy.editor import ImageSequenceClip, AudioFileClip, concatenate_videoclips

# def create_video_from_frames(frames, audio_file=None, video_file="final_video.mp4"):
#     if not frames:
#         raise ValueError("No frames to create video from.")
    
#     video_clips = [ImageSequenceClip([np.array(frame)], fps=24) for frame in frames]
    
#     video = video_clips[0]
#     for clip in video_clips[1:]:
#         video = concatenate_videoclips([video, clip], method="compose")
    
#     if audio_file and os.path.isfile(audio_file):
#         audio = AudioFileClip(audio_file)
#         video = video.set_audio(audio)
    
#     video.write_videofile(video_file, codec="libx264", fps=24)

In [24]:
# def generate_infographic_video(data, insights, audio_file=None, title_image="title_screen.png"):
#     frames = generate_default_frames(data)
#     if not frames:
#         raise ValueError("No frames generated from data.")
    
#     if os.path.exists(title_image):
#         title_image_clip = Image.open(title_image)
#         frames.insert(0, title_image_clip)
    
#     create_video_from_frames(frames, audio_file)
#     print("Video successfully generated!")

# def data_storytelling_pipeline(file_path, prompt, audio_file=None):
#     try:
#         start_time = time.time()
        
#         logging.info("Loading and preprocessing data...")
#         data = load_and_preprocess_data(file_path)
#         logging.debug(f"Loaded Data: {data.head()}")
        
#         logging.info("Performing EDA...")
#         eda_summary = perform_eda(data)
#         logging.debug(f"EDA Summary: {eda_summary}")
        
#         logging.info("Analyzing the user's prompt...")
#         insights = analyze_prompt_for_insights(prompt)
#         logging.debug(f"Extracted insights: {insights}")
        
#         logging.info("Creating the infographic video...")
#         generate_infographic_video(data, insights, audio_file=audio_file)
        
#         end_time = time.time()
#         logging.info(f"Pipeline completed successfully in {end_time - start_time:.2f} seconds")
    
#     except FileNotFoundError as fnf_error:
#         logging.error(f"File not found: {fnf_error}")
#         raise
#     except pd.errors.ParserError as parser_error:
#         logging.error(f"Error parsing the file: {parser_error}")
#         raise
#     except TypeError as type_error:
#         logging.error(f"Type error: {type_error}")
#         raise
#     except ValueError as value_error:
#         logging.error(f"Value error: {value_error}")
#         raise
#     except Exception as e:
#         logging.error(f"An unexpected error occurred: {e}")
#         raise

In [25]:
# # Example usage:
# file_path = '/kaggle/input/model-dataset-new-1/2015.csv'
# prompt = "Analyze the country and region data"
# audio_file = "/kaggle/working/"
# title_image = "/kaggle/working/"

# if not os.path.exists(file_path):
#     raise FileNotFoundError(f"Data file not found: {file_path}")
# if not os.path.exists(audio_file):
#     raise FileNotFoundError(f"Audio file not found: {audio_file}")
# if not os.path.exists(title_image):
#     raise FileNotFoundError(f"Title image not found: {title_image}")

# data_storytelling_pipeline(file_path, prompt, audio_file=audio_file)
# # # Predefined prompts for prompt engineering
# # predefined_prompts = [
# #     "Analyze the sales trends over time and regional performance.",
# #     "Identify key trends, comparisons, distributions, and correlations in the data.",
# #     "Generate insights on the performance and growth of different regions.",
# #     "Highlight any anomalies or outliers in the sales data.",
# #     "Compare the sales performance across different product categories.",
# #     "Analyze the correlation between sales and marketing spend.",
# #     "Identify patterns in customer purchase behavior.",
# #     "Generate insights on the distribution of sales across different regions.",
# #     "Analyze the relationship between sales and customer demographics.",
# #     "Identify key performance indicators for the sales team.",
# #     # Add more prompts as needed
# # ]

# # # Test the pipeline with predefined prompts
# # for test_prompt in predefined_prompts:
# #     try:
# #         logging.info(f"Testing with prompt: {test_prompt}")
# #         data_storytelling_pipeline(file_path, test_prompt, audio_file=audio_file)
# #     except Exception as e:
# #         logging.error(f"Error with prompt '{test_prompt}': {e}")

In [26]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# import logging
# import os
# from PIL import Image
# from moviepy.editor import ImageSequenceClip, AudioFileClip, concatenate_videoclips
# import numpy as np

# def generate_default_frames(data):
#     logging.info("Generating default frames from data")
#     frames = []
    
#     # Generate a basic time series plot if a date column is present
#     date_column = None
#     for col in data.columns:
#         if 'date' in col.lower() or 'time' in col.lower():
#             date_column = col
#             break
    
#     if date_column:
#         numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns
#         if len(numeric_columns) > 0:
#             plt.figure(figsize=(10, 6))
#             for col in numeric_columns:
#                 plt.plot(data[date_column], data[col], label=col)
#             plt.title(f"Time Series Plot for {', '.join(numeric_columns)}")
#             plt.xlabel('Time')
#             plt.ylabel('Values')
#             plt.xticks(rotation=45)
#             plt.legend()
#             frames.append(plt)
    
#     # Generate a correlation matrix plot
#     if data.select_dtypes(include=['float64', 'int64']).shape[1] > 1:
#         corr_matrix = data.corr()
#         plt.figure(figsize=(10, 6))
#         sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
#         plt.title('Correlation Matrix')
#         frames.append(plt)
    
#     # Generate distribution plots for numeric columns
#     numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns
#     for col in numeric_columns:
#         plt.figure(figsize=(8, 6))
#         sns.histplot(data[col], bins=10, kde=True)
#         plt.title(f"Distribution of {col}")
#         frames.append(plt)
    
#     logging.info(f"Generated {len(frames)} default frames")
#     return frames

# def create_video_from_frames(frames, audio_file=None, video_file="final_video.mp4"):
#     if not frames:
#         raise ValueError("No frames to create video from.")
    
#     logging.info("Creating video from frames")
#     video_clips = [ImageSequenceClip([np.array(frame.canvas.buffer_rgba())], fps=24) for frame in frames]
    
#     video = video_clips[0]
#     for clip in video_clips[1:]:
#         video = concatenate_videoclips([video, clip], method="compose")
    
#     if audio_file and os.path.isfile(audio_file):
#         audio = AudioFileClip(audio_file)
#         video = video.set_audio(audio)
    
#     video.write_videofile(video_file, codec="libx264", fps=24)
#     logging.info(f"Video saved as {video_file}")

# def generate_infographic_video(data, insights, audio_file=None, title_image="title_screen.png"):
#     frames = generate_default_frames(data)
#     if not frames:
#         raise ValueError("No frames generated from data.")
    
#     if os.path.exists(title_image):
#         title_image_clip = Image.open(title_image)
#         title_image_clip = title_image_clip.convert("RGBA")
#         title_image_clip = np.array(title_image_clip)
#         frames.insert(0, title_image_clip)
    
#     create_video_from_frames(frames, audio_file)
#     print("Video successfully generated!")

# def data_storytelling_pipeline(file_path, prompt, audio_file=None):
#     try:
#         start_time = time.time()
        
#         logging.info("Loading and preprocessing data...")
#         data = load_and_preprocess_data(file_path)
#         logging.debug(f"Loaded Data: {data.head()}")
        
#         logging.info("Performing EDA...")
#         eda_summary = perform_eda(data)
#         logging.debug(f"EDA Summary: {eda_summary}")
        
#         logging.info("Analyzing the user's prompt...")
#         insights = analyze_prompt_for_insights(prompt)
#         logging.debug(f"Extracted insights: {insights}")
        
#         logging.info("Creating the infographic video...")
#         generate_infographic_video(data, insights, audio_file=audio_file)
        
#         end_time = time.time()
#         logging.info(f"Pipeline completed successfully in {end_time - start_time:.2f} seconds")
    
#     except FileNotFoundError as fnf_error:
#         logging.error(f"File not found: {fnf_error}")
#         raise
#     except pd.errors.ParserError as parser_error:
#         logging.error(f"Error parsing the file: {parser_error}")
#         raise
#     except TypeError as type_error:
#         logging.error(f"Type error: {type_error}")
#         raise
#     except ValueError as value_error:
#         logging.error(f"Value error: {value_error}")
#         raise
#     except Exception as e:
#         logging.error(f"An unexpected error occurred: {e}")
#         raise

# # Example usage:
# file_path = '/kaggle/input/model-dataset-new-1/2015.csv'
# prompt = "Analyze the country and region data"
# audio_file = "/kaggle/working/"
# title_image = "/kaggle/working/"

# if not os.path.exists(file_path):
#     raise FileNotFoundError(f"Data file not found: {file_path}")
# if not os.path.exists(audio_file):
#     raise FileNotFoundError(f"Audio file not found: {audio_file}")
# if not os.path.exists(title_image):
#     raise FileNotFoundError(f"Title image not found: {title_image}")

# data_storytelling_pipeline(file_path, prompt, audio_file=audio_file)

# OLD MODEL CODE IMPLEMENTATION TILL HERE 

# WORKING CODE COMMENTED FOR REFERANCE 

In [27]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# import logging
# import os
# from PIL import Image
# from moviepy.editor import ImageSequenceClip, AudioFileClip, concatenate_videoclips
# import numpy as np
# import io

# def generate_default_frames(data):
#     logging.info("Generating default frames from data")
#     frames = []
    
#     # Generate a basic time series plot if a date column is present
#     date_column = None
#     for col in data.columns:
#         if 'date' in col.lower() or 'time' in col.lower():
#             date_column = col
#             break
    
#     if date_column:
#         numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns
#         if len(numeric_columns) > 0:
#             fig, ax = plt.subplots(figsize=(10, 6))
#             for col in numeric_columns:
#                 ax.plot(data[date_column], data[col], label=col)
#             ax.set_title(f"Time Series Plot for {', '.join(numeric_columns)}")
#             ax.set_xlabel('Time')
#             ax.set_ylabel('Values')
#             ax.legend()
#             buf = io.BytesIO()
#             fig.savefig(buf, format='png')
#             buf.seek(0)
#             img = Image.open(buf)
#             frames.append(img)
#             plt.close(fig)
    
#     # Generate a correlation matrix plot
#     if data.select_dtypes(include=['float64', 'int64']).shape[1] > 1:
#         corr_matrix = data.corr()
#         fig, ax = plt.subplots(figsize=(10, 6))
#         sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5, ax=ax)
#         ax.set_title('Correlation Matrix')
#         buf = io.BytesIO()
#         fig.savefig(buf, format='png')
#         buf.seek(0)
#         img = Image.open(buf)
#         frames.append(img)
#         plt.close(fig)
    
#     # Generate distribution plots for numeric columns
#     numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns
#     for col in numeric_columns:
#         fig, ax = plt.subplots(figsize=(8, 6))
#         sns.histplot(data[col], bins=10, kde=True, ax=ax)
#         ax.set_title(f"Distribution of {col}")
#         buf = io.BytesIO()
#         fig.savefig(buf, format='png')
#         buf.seek(0)
#         img = Image.open(buf)
#         frames.append(img)
#         plt.close(fig)
    
#     logging.info(f"Generated {len(frames)} default frames")
#     return frames

# def create_video_from_frames(frames, audio_file=None, video_file="final_video.mp4"):
#     if not frames:
#         raise ValueError("No frames to create video from.")
    
#     logging.info("Creating video from frames")
#     video_clips = [ImageSequenceClip([np.array(frame)], fps=24) for frame in frames]
    
#     video = video_clips[0]
#     for clip in video_clips[1:]:
#         video = concatenate_videoclips([video, clip], method="compose")
    
#     if audio_file and os.path.isfile(audio_file):
#         audio = AudioFileClip(audio_file)
#         video = video.set_audio(audio)
    
#     video.write_videofile(video_file, codec="libx264", fps=24)
#     logging.info(f"Video saved as {video_file}")

# def generate_infographic_video(data, insights, audio_file=None, title_image="title_screen.png"):
#     frames = generate_default_frames(data)
#     if not frames:
#         raise ValueError("No frames generated from data.")
    
#     if os.path.exists(title_image):
#         title_image_clip = Image.open(title_image)
#         title_image_clip = title_image_clip.convert("RGBA")
#         title_image_clip = np.array(title_image_clip)
#         frames.insert(0, title_image_clip)
    
#     create_video_from_frames(frames, audio_file)
#     print("Video successfully generated!")

# def data_storytelling_pipeline(file_path, prompt, audio_file=None):
#     try:
#         start_time = time.time()
        
#         logging.info("Loading and preprocessing data...")
#         data = load_and_preprocess_data(file_path)
#         logging.debug(f"Loaded Data: {data.head()}")
        
#         logging.info("Performing EDA...")
#         eda_summary = perform_eda(data)
#         logging.debug(f"EDA Summary: {eda_summary}")
        
#         logging.info("Analyzing the user's prompt...")
#         insights = analyze_prompt_for_insights(prompt)
#         logging.debug(f"Extracted insights: {insights}")
        
#         logging.info("Creating the infographic video...")
#         generate_infographic_video(data, insights, audio_file=audio_file)
        
#         end_time = time.time()
#         logging.info(f"Pipeline completed successfully in {end_time - start_time:.2f} seconds")
    
#     except FileNotFoundError as fnf_error:
#         logging.error(f"File not found: {fnf_error}")
#         raise
#     except pd.errors.ParserError as parser_error:
#         logging.error(f"Error parsing the file: {parser_error}")
#         raise
#     except TypeError as type_error:
#         logging.error(f"Type error: {type_error}")
#         raise
#     except ValueError as value_error:
#         logging.error(f"Value error: {value_error}")
#         raise
#     except Exception as e:
#         logging.error(f"An unexpected error occurred: {e}")
#         raise

# # Example usage:
# file_path = '/kaggle/input/model-dataset-new-1/2015.csv'
# prompt = "Analyze the country and region data"
# audio_file = "/kaggle/working/"
# title_image = "/kaggle/working/"

# if not os.path.exists(file_path):
#     raise FileNotFoundError(f"Data file not found: {file_path}")
# if not os.path.exists(audio_file):
#     raise FileNotFoundError(f"Audio file not found: {audio_file}")
# if not os.path.exists(title_image):
#     raise FileNotFoundError(f"Title image not found: {title_image}")

# data_storytelling_pipeline(file_path, prompt, audio_file=audio_file)

In [28]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# import logging
# import os
# from PIL import Image
# from moviepy.editor import ImageSequenceClip, AudioFileClip, concatenate_videoclips
# import numpy as np
# import io
# from matplotlib.animation import FuncAnimation

# def generate_animated_frames(data):
#     logging.info("Generating animated frames from data")
#     frames = []
    
#     # Generate a basic time series plot if a date column is present
#     date_column = None
#     for col in data.columns:
#         if 'date' in col.lower() or 'time' in col.lower():
#             date_column = col
#             break
    
#     if date_column:
#         numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns
#         if len(numeric_columns) > 0:
#             fig, ax = plt.subplots(figsize=(10, 6))
#             lines = [ax.plot([], [], label=col)[0] for col in numeric_columns]
#             ax.set_xlim(data[date_column].min(), data[date_column].max())
#             ax.set_ylim(data[numeric_columns].min().min(), data[numeric_columns].max().max())
#             ax.set_title(f"Time Series Plot for {', '.join(numeric_columns)}")
#             ax.set_xlabel('Time')
#             ax.set_ylabel('Values')
#             ax.legend()

#             def update(frame):
#                 for line, col in zip(lines, numeric_columns):
#                     line.set_data(data[date_column][:frame], data[col][:frame])
#                 return lines

#             ani = FuncAnimation(fig, update, frames=len(data), blit=True)
#             for i in range(len(data)):
#                 update(i)
#                 buf = io.BytesIO()
#                 fig.savefig(buf, format='png')
#                 buf.seek(0)
#                 img = Image.open(buf)
#                 frames.append(img)
#             plt.close(fig)
    
#     # Generate a correlation matrix plot
#     if data.select_dtypes(include=['float64', 'int64']).shape[1] > 1:
#         corr_matrix = data.corr()
#         fig, ax = plt.subplots(figsize=(10, 6))
#         sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5, ax=ax)
#         ax.set_title('Correlation Matrix')
#         buf = io.BytesIO()
#         fig.savefig(buf, format='png')
#         buf.seek(0)
#         img = Image.open(buf)
#         frames.append(img)
#         plt.close(fig)
    
#     # Generate distribution plots for numeric columns
#     numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns
#     for col in numeric_columns:
#         fig, ax = plt.subplots(figsize=(8, 6))
#         sns.histplot(data[col], bins=10, kde=True, ax=ax)
#         ax.set_title(f"Distribution of {col}")
#         buf = io.BytesIO()
#         fig.savefig(buf, format='png')
#         buf.seek(0)
#         img = Image.open(buf)
#         frames.append(img)
#         plt.close(fig)
    
#     logging.info(f"Generated {len(frames)} animated frames")
#     return frames

# def create_video_from_frames(frames, audio_file=None, video_file="final_video.mp4"):
#     if not frames:
#         raise ValueError("No frames to create video from.")
    
#     logging.info("Creating video from frames")
#     video_clips = [ImageSequenceClip([np.array(frame)], fps=24) for frame in frames]
    
#     video = video_clips[0]
#     for clip in video_clips[1:]:
#         video = concatenate_videoclips([video, clip], method="compose")
    
#     if audio_file and os.path.isfile(audio_file):
#         audio = AudioFileClip(audio_file)
#         video = video.set_audio(audio)
    
#     video.write_videofile(video_file, codec="libx264", fps=24)
#     logging.info(f"Video saved as {video_file}")

# def generate_infographic_video(data, insights, audio_file=None, title_image="title_screen.png"):
#     frames = generate_animated_frames(data)
#     if not frames:
#         raise ValueError("No frames generated from data.")
    
#     if os.path.exists(title_image):
#         title_image_clip = Image.open(title_image)
#         title_image_clip = title_image_clip.convert("RGBA")
#         title_image_clip = np.array(title_image_clip)
#         frames.insert(0, title_image_clip)
    
#     create_video_from_frames(frames, audio_file)
#     print("Video successfully generated!")

# def data_storytelling_pipeline(file_path, prompt, audio_file=None):
#     try:
#         start_time = time.time()
        
#         logging.info("Loading and preprocessing data...")
#         data = load_and_preprocess_data(file_path)
#         logging.debug(f"Loaded Data: {data.head()}")
        
#         logging.info("Performing EDA...")
#         eda_summary = perform_eda(data)
#         logging.debug(f"EDA Summary: {eda_summary}")
        
#         logging.info("Analyzing the user's prompt...")
#         insights = analyze_prompt_for_insights(prompt)
#         logging.debug(f"Extracted insights: {insights}")
        
#         logging.info("Creating the infographic video...")
#         generate_infographic_video(data, insights, audio_file=audio_file)
        
#         end_time = time.time()
#         logging.info(f"Pipeline completed successfully in {end_time - start_time:.2f} seconds")
    
#     except FileNotFoundError as fnf_error:
#         logging.error(f"File not found: {fnf_error}")
#         raise
#     except pd.errors.ParserError as parser_error:
#         logging.error(f"Error parsing the file: {parser_error}")
#         raise
#     except TypeError as type_error:
#         logging.error(f"Type error: {type_error}")
#         raise
#     except ValueError as value_error:
#         logging.error(f"Value error: {value_error}")
#         raise
#     except Exception as e:
#         logging.error(f"An unexpected error occurred: {e}")
#         raise

# # Example usage:
# file_path = '/kaggle/input/model-dataset-new-1/2015.csv'
# prompt = "Analyze the country and region data"
# audio_file = "/kaggle/working/"
# title_image = "/kaggle/working/"

# if not os.path.exists(file_path):
#     raise FileNotFoundError(f"Data file not found: {file_path}")
# if not os.path.exists(audio_file):
#     raise FileNotFoundError(f"Audio file not found: {audio_file}")
# if not os.path.exists(title_image):
#     raise FileNotFoundError(f"Title image not found: {title_image}")

# data_storytelling_pipeline(file_path, prompt, audio_file=audio_file)

In [29]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# import logging
# import os
# from PIL import Image
# from moviepy.editor import ImageSequenceClip, AudioFileClip, concatenate_videoclips, CompositeVideoClip
# import numpy as np
# import io
# from matplotlib.animation import FuncAnimation

# def generate_animated_frames(data):
#     logging.info("Generating animated frames from data")
#     frames = []
    
#     # Generate a basic time series plot if a date column is present
#     date_column = None
#     for col in data.columns:
#         if 'date' in col.lower() or 'time' in col.lower():
#             date_column = col
#             break
    
#     if date_column:
#         numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns
#         if len(numeric_columns) > 0:
#             fig, ax = plt.subplots(figsize=(10, 6))
#             lines = [ax.plot([], [], label=col)[0] for col in numeric_columns]
#             ax.set_xlim(data[date_column].min(), data[date_column].max())
#             ax.set_ylim(data[numeric_columns].min().min(), data[numeric_columns].max().max())
#             ax.set_title(f"Time Series Plot for {', '.join(numeric_columns)}")
#             ax.set_xlabel('Time')
#             ax.set_ylabel('Values')
#             ax.legend()

#             def update(frame):
#                 for line, col in zip(lines, numeric_columns):
#                     line.set_data(data[date_column][:frame], data[col][:frame])
#                 return lines

#             ani = FuncAnimation(fig, update, frames=len(data), blit=True)
#             for i in range(len(data)):
#                 update(i)
#                 buf = io.BytesIO()
#                 fig.savefig(buf, format='png')
#                 buf.seek(0)
#                 img = Image.open(buf)
#                 frames.append(img)
#             plt.close(fig)
    
#     # Generate a correlation matrix plot
#     if data.select_dtypes(include=['float64', 'int64']).shape[1] > 1:
#         corr_matrix = data.corr()
#         fig, ax = plt.subplots(figsize=(10, 6))
#         sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5, ax=ax)
#         ax.set_title('Correlation Matrix')
#         buf = io.BytesIO()
#         fig.savefig(buf, format='png')
#         buf.seek(0)
#         img = Image.open(buf)
#         frames.append(img)
#         plt.close(fig)
    
#     # Generate distribution plots for numeric columns
#     numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns
#     for col in numeric_columns:
#         fig, ax = plt.subplots(figsize=(8, 6))
#         sns.histplot(data[col], bins=10, kde=True, ax=ax)
#         ax.set_title(f"Distribution of {col}")
#         buf = io.BytesIO()
#         fig.savefig(buf, format='png')
#         buf.seek(0)
#         img = Image.open(buf)
#         frames.append(img)
#         plt.close(fig)
    
#     logging.info(f"Generated {len(frames)} animated frames")
#     return frames

# def create_video_from_frames(frames, audio_file=None, video_file="final_video.mp4"):
#     if not frames:
#         raise ValueError("No frames to create video from.")
    
#     logging.info("Creating video from frames")
#     video_clips = [ImageSequenceClip([np.array(frame)], fps=24) for frame in frames]
    
#     # Add transitions between clips
#     def add_transition(clip1, clip2, duration=1):
#         return concatenate_videoclips([clip1.crossfadeout(duration), clip2.crossfadein(duration)], method="compose")
    
#     video = video_clips[0]
#     for clip in video_clips[1:]:
#         video = add_transition(video, clip)
    
#     if audio_file and os.path.isfile(audio_file):
#         audio = AudioFileClip(audio_file)
#         video = video.set_audio(audio)
    
#     video = video.set_duration(max(60, video.duration))  # Ensure video is at least 60 seconds long
#     video.write_videofile(video_file, codec="libx264", fps=24)
#     logging.info(f"Video saved as {video_file}")

# def generate_infographic_video(data, insights, audio_file=None, title_image="title_screen.png"):
#     frames = generate_animated_frames(data)
#     if not frames:
#         raise ValueError("No frames generated from data.")
    
#     if os.path.exists(title_image):
#         title_image_clip = Image.open(title_image)
#         title_image_clip = title_image_clip.convert("RGBA")
#         title_image_clip = np.array(title_image_clip)
#         frames.insert(0, title_image_clip)
    
#     create_video_from_frames(frames, audio_file)
#     print("Video successfully generated!")

# def data_storytelling_pipeline(file_path, prompt, audio_file=None):
#     try:
#         start_time = time.time()
        
#         logging.info("Loading and preprocessing data...")
#         data = load_and_preprocess_data(file_path)
#         logging.debug(f"Loaded Data: {data.head()}")
        
#         logging.info("Performing EDA...")
#         eda_summary = perform_eda(data)
#         logging.debug(f"EDA Summary: {eda_summary}")
        
#         logging.info("Analyzing the user's prompt...")
#         insights = analyze_prompt_for_insights(prompt)
#         logging.debug(f"Extracted insights: {insights}")
        
#         logging.info("Creating the infographic video...")
#         generate_infographic_video(data, insights, audio_file=audio_file)
        
#         end_time = time.time()
#         logging.info(f"Pipeline completed successfully in {end_time - start_time:.2f} seconds")
    
#     except FileNotFoundError as fnf_error:
#         logging.error(f"File not found: {fnf_error}")
#         raise
#     except pd.errors.ParserError as parser_error:
#         logging.error(f"Error parsing the file: {parser_error}")
#         raise
#     except TypeError as type_error:
#         logging.error(f"Type error: {type_error}")
#         raise
#     except ValueError as value_error:
#         logging.error(f"Value error: {value_error}")
#         raise
#     except Exception as e:
#         logging.error(f"An unexpected error occurred: {e}")
#         raise

# # Example usage:
# file_path = '/kaggle/input/model-dataset-new-1/2015.csv'
# prompt = "Analyze the country and region data"
# audio_file = "/kaggle/working/"
# title_image = "/kaggle/working/"

# if not os.path.exists(file_path):
#     raise FileNotFoundError(f"Data file not found: {file_path}")
# if not os.path.exists(audio_file):
#     raise FileNotFoundError(f"Audio file not found: {audio_file}")
# if not os.path.exists(title_image):
#     raise FileNotFoundError(f"Title image not found: {title_image}")

# data_storytelling_pipeline(file_path, prompt, audio_file=audio_file)

In [30]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# import logging
# import os
# from PIL import Image
# from moviepy.editor import ImageSequenceClip, AudioFileClip, concatenate_videoclips, CompositeVideoClip, TextClip
# import numpy as np
# import io
# from matplotlib.animation import FuncAnimation

# def generate_animated_frames(data):
#     logging.info("Generating animated frames from data")
#     frames = []
    
#     # Generate a basic time series plot if a date column is present
#     date_column = None
#     for col in data.columns:
#         if 'date' in col.lower() or 'time' in col.lower():
#             date_column = col
#             break
    
#     if date_column:
#         numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns
#         if len(numeric_columns) > 0:
#             fig, ax = plt.subplots(figsize=(10, 6))
#             lines = [ax.plot([], [], label=col)[0] for col in numeric_columns]
#             ax.set_xlim(data[date_column].min(), data[date_column].max())
#             ax.set_ylim(data[numeric_columns].min().min(), data[numeric_columns].max().max())
#             ax.set_title(f"Time Series Plot for {', '.join(numeric_columns)}")
#             ax.set_xlabel('Time')
#             ax.set_ylabel('Values')
#             ax.legend()

#             def update(frame):
#                 for line, col in zip(lines, numeric_columns):
#                     line.set_data(data[date_column][:frame], data[col][:frame])
#                 return lines

#             ani = FuncAnimation(fig, update, frames=len(data), blit=True)
#             for i in range(len(data)):
#                 update(i)
#                 buf = io.BytesIO()
#                 fig.savefig(buf, format='png')
#                 buf.seek(0)
#                 img = Image.open(buf)
#                 frames.append(img)
#             plt.close(fig)
    
#     # Generate a correlation matrix plot
#     if data.select_dtypes(include=['float64', 'int64']).shape[1] > 1:
#         corr_matrix = data.corr()
#         fig, ax = plt.subplots(figsize=(10, 6))
#         sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5, ax=ax)
#         ax.set_title('Correlation Matrix')
#         buf = io.BytesIO()
#         fig.savefig(buf, format='png')
#         buf.seek(0)
#         img = Image.open(buf)
#         frames.append(img)
#         plt.close(fig)
    
#     # Generate distribution plots for numeric columns
#     numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns
#     for col in numeric_columns:
#         fig, ax = plt.subplots(figsize=(8, 6))
#         sns.histplot(data[col], bins=10, kde=True, ax=ax)
#         ax.set_title(f"Distribution of {col}")
#         buf = io.BytesIO()
#         fig.savefig(buf, format='png')
#         buf.seek(0)
#         img = Image.open(buf)
#         frames.append(img)
#         plt.close(fig)
    
#     logging.info(f"Generated {len(frames)} animated frames")
#     return frames

# def create_video_from_frames(frames, audio_file=None, video_file="final_video.mp4"):
#     if not frames:
#         raise ValueError("No frames to create video from.")
    
#     logging.info("Creating video from frames")
#     video_clips = [ImageSequenceClip([np.array(frame)], fps=24) for frame in frames]
    
#     # Add transitions between clips
#     def add_transition(clip1, clip2, duration=1):
#         return concatenate_videoclips([clip1.crossfadeout(duration), clip2.crossfadein(duration)], method="compose")
    
#     video = video_clips[0]
#     for clip in video_clips[1:]:
#         video = add_transition(video, clip)
    
#     if audio_file and os.path.isfile(audio_file):
#         audio = AudioFileClip(audio_file)
#         video = video.set_audio(audio)
    
#     # Add a title screen
#     title = TextClip("Data Storytelling", fontsize=70, color='white', bg_color='black', size=video.size)
#     title = title.set_duration(3).fadein(1).fadeout(1)
#     video = concatenate_videoclips([title, video])
    
#     video = video.set_duration(max(60, video.duration))  # Ensure video is at least 60 seconds long
#     video.write_videofile(video_file, codec="libx264", fps=24)
#     logging.info(f"Video saved as {video_file}")

# def generate_infographic_video(data, insights, audio_file=None, title_image="title_screen.png"):
#     frames = generate_animated_frames(data)
#     if not frames:
#         raise ValueError("No frames generated from data.")
    
#     if os.path.exists(title_image):
#         title_image_clip = Image.open(title_image)
#         title_image_clip = title_image_clip.convert("RGBA")
#         title_image_clip = np.array(title_image_clip)
#         frames.insert(0, title_image_clip)
    
#     create_video_from_frames(frames, audio_file)
#     print("Video successfully generated!")

# def data_storytelling_pipeline(file_path, prompt, audio_file=None):
#     try:
#         start_time = time.time()
        
#         logging.info("Loading and preprocessing data...")
#         data = load_and_preprocess_data(file_path)
#         logging.debug(f"Loaded Data: {data.head()}")
        
#         logging.info("Performing EDA...")
#         eda_summary = perform_eda(data)
#         logging.debug(f"EDA Summary: {eda_summary}")
        
#         logging.info("Analyzing the user's prompt...")
#         insights = analyze_prompt_for_insights(prompt)
#         logging.debug(f"Extracted insights: {insights}")
        
#         logging.info("Creating the infographic video...")
#         generate_infographic_video(data, insights, audio_file=audio_file)
        
#         end_time = time.time()
#         logging.info(f"Pipeline completed successfully in {end_time - start_time:.2f} seconds")
    
#     except FileNotFoundError as fnf_error:
#         logging.error(f"File not found: {fnf_error}")
#         raise
#     except pd.errors.ParserError as parser_error:
#         logging.error(f"Error parsing the file: {parser_error}")
#         raise
#     except TypeError as type_error:
#         logging.error(f"Type error: {type_error}")
#         raise
#     except ValueError as value_error:
#         logging.error(f"Value error: {value_error}")
#         raise
#     except Exception as e:
#         logging.error(f"An unexpected error occurred: {e}")
#         raise

# # Example usage:
# file_path = '/kaggle/input/model-dataset-new-1/2015.csv'
# prompt = "Analyze the country and region data"
# audio_file = "/kaggle/working/"
# title_image = "/kaggle/working/"

# if not os.path.exists(file_path):
#     raise FileNotFoundError(f"Data file not found: {file_path}")
# if not os.path.exists(audio_file):
#     raise FileNotFoundError(f"Audio file not found: {audio_file}")
# if not os.path.exists(title_image):
#     raise FileNotFoundError(f"Title image not found: {title_image}")

# data_storytelling_pipeline(file_path, prompt, audio_file=audio_file)

In [31]:
# !pip install imagemagick

In [32]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# import logging
# import os
# from PIL import Image
# from moviepy.editor import ImageSequenceClip, AudioFileClip, concatenate_videoclips, CompositeVideoClip, TextClip
# import numpy as np
# import io
# from matplotlib.animation import FuncAnimation

# def generate_animated_frames(data):
#     logging.info("Generating animated frames from data")
#     frames = []
    
#     # Generate a basic time series plot if a date column is present
#     date_column = None
#     for col in data.columns:
#         if 'date' in col.lower() or 'time' in col.lower():
#             date_column = col
#             break
    
#     if date_column:
#         numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns
#         if len(numeric_columns) > 0:
#             fig, ax = plt.subplots(figsize=(10, 6))
#             lines = [ax.plot([], [], label=col)[0] for col in numeric_columns]
#             ax.set_xlim(data[date_column].min(), data[date_column].max())
#             ax.set_ylim(data[numeric_columns].min().min(), data[numeric_columns].max().max())
#             ax.set_title(f"Time Series Plot for {', '.join(numeric_columns)}")
#             ax.set_xlabel('Time')
#             ax.set_ylabel('Values')
#             ax.legend()

#             def update(frame):
#                 for line, col in zip(lines, numeric_columns):
#                     line.set_data(data[date_column][:frame], data[col][:frame])
#                 return lines

#             ani = FuncAnimation(fig, update, frames=len(data), blit=True)
#             for i in range(len(data)):
#                 update(i)
#                 buf = io.BytesIO()
#                 fig.savefig(buf, format='png')
#                 buf.seek(0)
#                 img = Image.open(buf)
#                 frames.append(img)
#             plt.close(fig)
    
#     # Generate a correlation matrix plot
#     if data.select_dtypes(include=['float64', 'int64']).shape[1] > 1:
#         corr_matrix = data.corr()
#         fig, ax = plt.subplots(figsize=(10, 6))
#         sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5, ax=ax)
#         ax.set_title('Correlation Matrix')
#         buf = io.BytesIO()
#         fig.savefig(buf, format='png')
#         buf.seek(0)
#         img = Image.open(buf)
#         frames.append(img)
#         plt.close(fig)
    
#     # Generate distribution plots for numeric columns
#     numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns
#     for col in numeric_columns:
#         fig, ax = plt.subplots(figsize=(8, 6))
#         sns.histplot(data[col], bins=10, kde=True, ax=ax)
#         ax.set_title(f"Distribution of {col}")
#         buf = io.BytesIO()
#         fig.savefig(buf, format='png')
#         buf.seek(0)
#         img = Image.open(buf)
#         frames.append(img)
#         plt.close(fig)
    
#     logging.info(f"Generated {len(frames)} animated frames")
#     return frames

# def create_video_from_frames(frames, audio_file=None, video_file="final_video.mp4"):
#     if not frames:
#         raise ValueError("No frames to create video from.")
    
#     logging.info("Creating video from frames")
#     video_clips = [ImageSequenceClip([np.array(frame)], fps=24) for frame in frames]
    
#     # Add transitions between clips
#     def add_transition(clip1, clip2, duration=1):
#         return concatenate_videoclips([clip1.crossfadeout(duration), clip2.crossfadein(duration)], method="compose")
    
#     video = video_clips[0]
#     for clip in video_clips[1:]:
#         video = add_transition(video, clip)
    
#     if audio_file and os.path.isfile(audio_file):
#         audio = AudioFileClip(audio_file)
#         video = video.set_audio(audio)
    
#     # Add a title screen
#     title = TextClip("Data Storytelling", fontsize=70, color='white', bg_color='black', size=video.size)
#     title = title.set_duration(3).fadein(1).fadeout(1)
#     video = concatenate_videoclips([title, video])
    
#     video = video.set_duration(max(60, video.duration))  # Ensure video is at least 60 seconds long
#     video.write_videofile(video_file, codec="libx264", fps=24)
#     logging.info(f"Video saved as {video_file}")

# def generate_infographic_video(data, insights, audio_file=None, title_image="title_screen.png"):
#     frames = generate_animated_frames(data)
#     if not frames:
#         raise ValueError("No frames generated from data.")
    
#     if os.path.exists(title_image):
#         title_image_clip = Image.open(title_image)
#         title_image_clip = title_image_clip.convert("RGBA")
#         title_image_clip = np.array(title_image_clip)
#         frames.insert(0, title_image_clip)
    
#     create_video_from_frames(frames, audio_file)
#     print("Video successfully generated!")

# def data_storytelling_pipeline(file_path, prompt, audio_file=None):
#     try:
#         start_time = time.time()
        
#         logging.info("Loading and preprocessing data...")
#         data = load_and_preprocess_data(file_path)
#         logging.debug(f"Loaded Data: {data.head()}")
        
#         logging.info("Performing EDA...")
#         eda_summary = perform_eda(data)
#         logging.debug(f"EDA Summary: {eda_summary}")
        
#         logging.info("Analyzing the user's prompt...")
#         insights = analyze_prompt_for_insights(prompt)
#         logging.debug(f"Extracted insights: {insights}")
        
#         logging.info("Creating the infographic video...")
#         generate_infographic_video(data, insights, audio_file=audio_file)
        
#         end_time = time.time()
#         logging.info(f"Pipeline completed successfully in {end_time - start_time:.2f} seconds")
    
#     except FileNotFoundError as fnf_error:
#         logging.error(f"File not found: {fnf_error}")
#         raise
#     except pd.errors.ParserError as parser_error:
#         logging.error(f"Error parsing the file: {parser_error}")
#         raise
#     except TypeError as type_error:
#         logging.error(f"Type error: {type_error}")
#         raise
#     except ValueError as value_error:
#         logging.error(f"Value error: {value_error}")
#         raise
#     except Exception as e:
#         logging.error(f"An unexpected error occurred: {e}")
#         raise

# # Example usage:
# file_path = '/kaggle/input/model-dataset-new-1/2015.csv'
# prompt = "Analyze the country and region data"
# audio_file = "/kaggle/working/"
# title_image = "/kaggle/working/"

# if not os.path.exists(file_path):
#     raise FileNotFoundError(f"Data file not found: {file_path}")
# if not os.path.exists(audio_file):
#     raise FileNotFoundError(f"Audio file not found: {audio_file}")
# if not os.path.exists(title_image):
#     raise FileNotFoundError(f"Title image not found: {title_image}")

# data_storytelling_pipeline(file_path, prompt, audio_file=audio_file)

In [33]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# import logging
# import os
# from PIL import Image
# import numpy as np
# import io
# from matplotlib.animation import FuncAnimation
# import cv2

# def generate_animated_frames(data):
#     logging.info("Generating animated frames from data")
#     frames = []
    
#     # Generate a basic time series plot if a date column is present
#     date_column = None
#     for col in data.columns:
#         if 'date' in col.lower() or 'time' in col.lower():
#             date_column = col
#             break
    
#     if date_column:
#         numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns
#         if len(numeric_columns) > 0:
#             fig, ax = plt.subplots(figsize=(10, 6))
#             lines = [ax.plot([], [], label=col)[0] for col in numeric_columns]
#             ax.set_xlim(data[date_column].min(), data[date_column].max())
#             ax.set_ylim(data[numeric_columns].min().min(), data[numeric_columns].max().max())
#             ax.set_title(f"Time Series Plot for {', '.join(numeric_columns)}")
#             ax.set_xlabel('Time')
#             ax.set_ylabel('Values')
#             ax.legend()

#             def update(frame):
#                 for line, col in zip(lines, numeric_columns):
#                     line.set_data(data[date_column][:frame], data[col][:frame])
#                 return lines

#             ani = FuncAnimation(fig, update, frames=len(data), blit=True)
#             for i in range(len(data)):
#                 update(i)
#                 buf = io.BytesIO()
#                 fig.savefig(buf, format='png')
#                 buf.seek(0)
#                 img = Image.open(buf)
#                 frames.append(np.array(img))
#             plt.close(fig)
    
#     # Generate a correlation matrix plot
#     if data.select_dtypes(include=['float64', 'int64']).shape[1] > 1:
#         corr_matrix = data.corr()
#         fig, ax = plt.subplots(figsize=(10, 6))
#         sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5, ax=ax)
#         ax.set_title('Correlation Matrix')
#         buf = io.BytesIO()
#         fig.savefig(buf, format='png')
#         buf.seek(0)
#         img = Image.open(buf)
#         frames.append(np.array(img))
#         plt.close(fig)
    
#     # Generate distribution plots for numeric columns
#     numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns
#     for col in numeric_columns:
#         fig, ax = plt.subplots(figsize=(8, 6))
#         sns.histplot(data[col], bins=10, kde=True, ax=ax)
#         ax.set_title(f"Distribution of {col}")
#         buf = io.BytesIO()
#         fig.savefig(buf, format='png')
#         buf.seek(0)
#         img = Image.open(buf)
#         frames.append(np.array(img))
#         plt.close(fig)
    
#     logging.info(f"Generated {len(frames)} animated frames")
#     return frames

# def create_video_from_frames(frames, audio_file=None, video_file="new_final_video.mp4"):
#     if not frames:
#         raise ValueError("No frames to create video from.")
    
#     logging.info("Creating video from frames")
#     height, width, layers = frames[0].shape
#     video = cv2.VideoWriter(video_file, cv2.VideoWriter_fourcc(*'mp4v'), 24, (width, height))
    
#     for frame in frames:
#         video.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
    
#     video.release()
#     logging.info(f"Video saved as {video_file}")

# def generate_infographic_video(data, insights, audio_file=None, title_image="title_screen.png"):
#     frames = generate_animated_frames(data)
#     if not frames:
#         raise ValueError("No frames generated from data.")
    
#     if os.path.exists(title_image):
#         title_image_clip = Image.open(title_image)
#         title_image_clip = title_image_clip.convert("RGBA")
#         title_image_clip = np.array(title_image_clip)
#         frames.insert(0, title_image_clip)
    
#     create_video_from_frames(frames, audio_file)
#     print("Video successfully generated!")

# def data_storytelling_pipeline(file_path, prompt, audio_file=None):
#     try:
#         start_time = time.time()
        
#         logging.info("Loading and preprocessing data...")
#         data = load_and_preprocess_data(file_path)
#         logging.debug(f"Loaded Data: {data.head()}")
        
#         logging.info("Performing EDA...")
#         eda_summary = perform_eda(data)
#         logging.debug(f"EDA Summary: {eda_summary}")
        
#         logging.info("Analyzing the user's prompt...")
#         insights = analyze_prompt_for_insights(prompt)
#         logging.debug(f"Extracted insights: {insights}")
        
#         logging.info("Creating the infographic video...")
#         generate_infographic_video(data, insights, audio_file=audio_file)
        
#         end_time = time.time()
#         logging.info(f"Pipeline completed successfully in {end_time - start_time:.2f} seconds")
    
#     except FileNotFoundError as fnf_error:
#         logging.error(f"File not found: {fnf_error}")
#         raise
#     except pd.errors.ParserError as parser_error:
#         logging.error(f"Error parsing the file: {parser_error}")
#         raise
#     except TypeError as type_error:
#         logging.error(f"Type error: {type_error}")
#         raise
#     except ValueError as value_error:
#         logging.error(f"Value error: {value_error}")
#         raise
#     except Exception as e:
#         logging.error(f"An unexpected error occurred: {e}")
#         raise

# # Example usage:
# file_path = '/kaggle/input/model-dataset-new-1/2015.csv'
# prompt = "Analyze the country and region data"
# audio_file = "/kaggle/working/"
# title_image = "/kaggle/working/"

# if not os.path.exists(file_path):
#     raise FileNotFoundError(f"Data file not found: {file_path}")
# if not os.path.exists(audio_file):
#     raise FileNotFoundError(f"Audio file not found: {audio_file}")
# if not os.path.exists(title_image):
#     raise FileNotFoundError(f"Title image not found: {title_image}")

# data_storytelling_pipeline(file_path, prompt, audio_file=audio_file)

In [34]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# import logging
# import os
# from PIL import Image
# import numpy as np
# import io
# from matplotlib.animation import FuncAnimation
# import cv2

# def generate_animated_frames(data):
#     logging.info("Generating animated frames from data")
#     frames = []
    
#     # Generate a basic time series plot if a date column is present
#     date_column = None
#     for col in data.columns:
#         if 'date' in col.lower() or 'time' in col.lower():
#             date_column = col
#             break
    
#     if date_column:
#         numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns
#         if len(numeric_columns) > 0:
#             fig, ax = plt.subplots(figsize=(10, 6))
#             lines = [ax.plot([], [], label=col)[0] for col in numeric_columns]
#             ax.set_xlim(data[date_column].min(), data[date_column].max())
#             ax.set_ylim(data[numeric_columns].min().min(), data[numeric_columns].max().max())
#             ax.set_title(f"Time Series Plot for {', '.join(numeric_columns)}")
#             ax.set_xlabel('Time')
#             ax.set_ylabel('Values')
#             ax.legend()

#             def update(frame):
#                 for line, col in zip(lines, numeric_columns):
#                     line.set_data(data[date_column][:frame], data[col][:frame])
#                 return lines

#             ani = FuncAnimation(fig, update, frames=len(data), blit=True)
#             for i in range(len(data)):
#                 update(i)
#                 buf = io.BytesIO()
#                 fig.savefig(buf, format='png')
#                 buf.seek(0)
#                 img = Image.open(buf)
#                 frames.append(np.array(img))
#             plt.close(fig)
    
#     # Generate a correlation matrix plot
#     if data.select_dtypes(include=['float64', 'int64']).shape[1] > 1:
#         corr_matrix = data.corr()
#         fig, ax = plt.subplots(figsize=(10, 6))
#         sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5, ax=ax)
#         ax.set_title('Correlation Matrix')
#         buf = io.BytesIO()
#         fig.savefig(buf, format='png')
#         buf.seek(0)
#         img = Image.open(buf)
#         frames.append(np.array(img))
#         plt.close(fig)
    
#     # Generate distribution plots for numeric columns
#     numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns
#     for col in numeric_columns:
#         fig, ax = plt.subplots(figsize=(8, 6))
#         sns.histplot(data[col], bins=10, kde=True, ax=ax)
#         ax.set_title(f"Distribution of {col}")
#         buf = io.BytesIO()
#         fig.savefig(buf, format='png')
#         buf.seek(0)
#         img = Image.open(buf)
#         frames.append(np.array(img))
#         plt.close(fig)
    
#     logging.info(f"Generated {len(frames)} animated frames")
#     return frames

# def create_video_from_frames(frames, audio_file=None, video_file="second_final_video.mp4"):
#     if not frames:
#         raise ValueError("No frames to create video from.")
    
#     logging.info("Creating video from frames")
#     height, width, layers = frames[0].shape
#     video = cv2.VideoWriter(video_file, cv2.VideoWriter_fourcc(*'mp4v'), 24, (width, height))
    
#     # Add a title screen
#     title_screen = np.zeros((height, width, 3), dtype=np.uint8)
#     title_screen.fill(0)  # Black background
#     cv2.putText(title_screen, "Data Storytelling", (50, height // 2), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 3)
#     for _ in range(72):  # 3 seconds at 24 fps
#         video.write(title_screen)
    
#     for frame in frames:
#         for _ in range(3):  # Repeat each frame to slow down the video
#             video.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
    
#     # Ensure video is at least 60 seconds long
#     while video.get(cv2.CAP_PROP_FRAME_COUNT) < 60 * 24:
#         video.write(title_screen)
    
#     video.release()
#     logging.info(f"Video saved as {video_file}")

# def generate_infographic_video(data, insights, audio_file=None, title_image="title_screen.png"):
#     frames = generate_animated_frames(data)
#     if not frames:
#         raise ValueError("No frames generated from data.")
    
#     if os.path.exists(title_image):
#         title_image_clip = Image.open(title_image)
#         title_image_clip = title_image_clip.convert("RGBA")
#         title_image_clip = np.array(title_image_clip)
#         frames.insert(0, title_image_clip)
    
#     create_video_from_frames(frames, audio_file)
#     print("Video successfully generated!")

# def data_storytelling_pipeline(file_path, prompt, audio_file=None):
#     try:
#         start_time = time.time()
        
#         logging.info("Loading and preprocessing data...")
#         data = load_and_preprocess_data(file_path)
#         logging.debug(f"Loaded Data: {data.head()}")
        
#         logging.info("Performing EDA...")
#         eda_summary = perform_eda(data)
#         logging.debug(f"EDA Summary: {eda_summary}")
        
#         logging.info("Analyzing the user's prompt...")
#         insights = analyze_prompt_for_insights(prompt)
#         logging.debug(f"Extracted insights: {insights}")
        
#         logging.info("Creating the infographic video...")
#         generate_infographic_video(data, insights, audio_file=audio_file)
        
#         end_time = time.time()
#         logging.info(f"Pipeline completed successfully in {end_time - start_time:.2f} seconds")
    
#     except FileNotFoundError as fnf_error:
#         logging.error(f"File not found: {fnf_error}")
#         raise
#     except pd.errors.ParserError as parser_error:
#         logging.error(f"Error parsing the file: {parser_error}")
#         raise
#     except TypeError as type_error:
#         logging.error(f"Type error: {type_error}")
#         raise
#     except ValueError as value_error:
#         logging.error(f"Value error: {value_error}")
#         raise
#     except Exception as e:
#         logging.error(f"An unexpected error occurred: {e}")
#         raise

# # Example usage:
# file_path = '/kaggle/input/model-dataset-new-1/2015.csv'
# prompt = "Analyze the country and region data"
# audio_file = "/kaggle/working/"
# title_image = "/kaggle/working/"

# if not os.path.exists(file_path):
#     raise FileNotFoundError(f"Data file not found: {file_path}")
# if not os.path.exists(audio_file):
#     raise FileNotFoundError(f"Audio file not found: {audio_file}")
# if not os.path.exists(title_image):
#     raise FileNotFoundError(f"Title image not found: {title_image}")

# data_storytelling_pipeline(file_path, prompt, audio_file=audio_file)

In [35]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# import logging
# import os
# from PIL import Image
# import numpy as np
# import io
# from matplotlib.animation import FuncAnimation
# import cv2
# import pandas as pd
# import time

# def generate_animated_frames(data):
#     logging.info("Generating animated frames from data")
#     frames = []
    
#     # Generate a basic time series plot if a date column is present
#     date_column = None
#     for col in data.columns:
#         if 'date' in col.lower() or 'time' in col.lower():
#             date_column = col
#             break
    
#     if date_column:
#         numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns
#         if len(numeric_columns) > 0:
#             fig, ax = plt.subplots(figsize=(10, 6))
#             lines = [ax.plot([], [], label=col)[0] for col in numeric_columns]
#             ax.set_xlim(data[date_column].min(), data[date_column].max())
#             ax.set_ylim(data[numeric_columns].min().min(), data[numeric_columns].max().max())
#             ax.set_title(f"Time Series Plot for {', '.join(numeric_columns)}")
#             ax.set_xlabel('Time')
#             ax.set_ylabel('Values')
#             ax.legend()

#             def update(frame):
#                 for line, col in zip(lines, numeric_columns):
#                     line.set_data(data[date_column][:frame], data[col][:frame])
#                 return lines

#             ani = FuncAnimation(fig, update, frames=len(data)//10, blit=True)
#             for i in range(len(data)//10):
#                 update(i)
#                 buf = io.BytesIO()
#                 fig.savefig(buf, format='png')
#                 buf.seek(0)
#                 img = Image.open(buf)
#                 frames.append(np.array(img))
#             plt.close(fig)
    
#     # Generate a moving bar chart
#     if 'category' in data.columns and 'value' in data.columns:
#         fig, ax = plt.subplots(figsize=(10, 6))
#         categories = data['category'].unique()
#         bars = ax.bar(categories, np.zeros(len(categories)))
#         ax.set_ylim(0, data['value'].max())
#         ax.set_title('Moving Bar Chart')
#         ax.set_xlabel('Category')
#         ax.set_ylabel('Value')

#         def update_bar(frame):
#             for bar, category in zip(bars, categories):
#                 bar.set_height(data[data['category'] == category]['value'].iloc[frame])
#             return bars

#         ani = FuncAnimation(fig, update_bar, frames=len(data)//10, blit=True)
#         for i in range(len(data)//10):
#             update_bar(i)
#             buf = io.BytesIO()
#             fig.savefig(buf, format='png')
#             buf.seek(0)
#             img = Image.open(buf)
#             frames.append(np.array(img))
#         plt.close(fig)
    
#     # Generate a correlation matrix plot
#     if data.select_dtypes(include=['float64', 'int64']).shape[1] > 1:
#         corr_matrix = data.corr()
#         fig, ax = plt.subplots(figsize=(10, 6))
#         sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5, ax=ax)
#         ax.set_title('Correlation Matrix')
#         buf = io.BytesIO()
#         fig.savefig(buf, format='png')
#         buf.seek(0)
#         img = Image.open(buf)
#         frames.append(np.array(img))
#         plt.close(fig)
    
#     # Generate distribution plots for numeric columns
#     numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns
#     for col in numeric_columns:
#         fig, ax = plt.subplots(figsize=(8, 6))
#         sns.histplot(data[col], bins=10, kde=True, ax=ax)
#         ax.set_title(f"Distribution of {col}")
#         buf = io.BytesIO()
#         fig.savefig(buf, format='png')
#         buf.seek(0)
#         img = Image.open(buf)
#         frames.append(np.array(img))
#         plt.close(fig)
    
#     # Generate a pairplot for numeric columns
#     if len(numeric_columns) > 1:
#         sns.pairplot(data[numeric_columns])
#         plt.suptitle('Pairplot of Numeric Columns', y=1.02)
#         buf = io.BytesIO()
#         plt.savefig(buf, format='png')
#         buf.seek(0)
#         img = Image.open(buf)
#         frames.append(np.array(img))
#         plt.close()
    
#     # Generate a line plot for each numeric column
#     for col in numeric_columns:
#         fig, ax = plt.subplots(figsize=(8, 6))
#         ax.plot(data.index, data[col])
#         ax.set_title(f"Line Plot of {col}")
#         ax.set_xlabel('Index')
#         ax.set_ylabel(col)
#         buf = io.BytesIO()
#         fig.savefig(buf, format='png')
#         buf.seek(0)
#         img = Image.open(buf)
#         frames.append(np.array(img))
#         plt.close(fig)
    
#     logging.info(f"Generated {len(frames)} animated frames")
#     return frames

# def create_video_from_frames(frames, audio_file=None, video_file="final_video.mp4"):
#     if not frames:
#         raise ValueError("No frames to create video from.")
    
#     logging.info("Creating video from frames")
#     height, width, layers = frames[0].shape
#     video = cv2.VideoWriter(video_file, cv2.VideoWriter_fourcc(*'mp4v'), 24, (width, height))
    
#     for frame in frames:
#         video.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
    
#     video.release()
#     logging.info(f"Video saved as {video_file}")

# def generate_infographic_video(data, insights, audio_file=None, title_image="title_screen.png"):
#     frames = generate_animated_frames(data)
#     if not frames:
#         raise ValueError("No frames generated from data.")
    
#     create_video_from_frames(frames, audio_file)
#     print("Video successfully generated!")

# def data_storytelling_pipeline(file_path, prompt, audio_file=None):
#     try:
#         start_time = time.time()
        
#         logging.info("Loading and preprocessing data...")
#         data_start_time = time.time()
#         data = load_and_preprocess_data(file_path)
#         logging.debug(f"Loaded Data: {data.head()}")
#         logging.info(f"Data loading and preprocessing took {time.time() - data_start_time:.2f} seconds")
        
#         logging.info("Performing EDA...")
#         eda_start_time = time.time()
#         eda_summary = perform_eda(data)
#         logging.debug(f"EDA Summary: {eda_summary}")
#         logging.info(f"EDA took {time.time() - eda_start_time:.2f} seconds")
        
#         logging.info("Analyzing the user's prompt...")
#         prompt_start_time = time.time()
#         insights = analyze_prompt_for_insights(prompt)
#         logging.debug(f"Extracted insights: {insights}")
#         logging.info(f"Prompt analysis took {time.time() - prompt_start_time:.2f} seconds")
        
#         logging.info("Creating the infographic video...")
#         video_start_time = time.time()
#         generate_infographic_video(data, insights, audio_file=audio_file)
#         logging.info(f"Video creation took {time.time() - video_start_time:.2f} seconds")
        
#         end_time = time.time()
#         logging.info(f"Pipeline completed successfully in {end_time - start_time:.2f} seconds")
    
#     except FileNotFoundError as fnf_error:
#         logging.error(f"File not found: {fnf_error}")
#         raise
#     except pd.errors.ParserError as parser_error:
#         logging.error(f"Error parsing the file: {parser_error}")
#         raise
#     except TypeError as type_error:
#         logging.error(f"Type error: {type_error}")
#         raise
#     except ValueError as value_error:
#         logging.error(f"Value error: {value_error}")
#         raise
#     except Exception as e:
#         logging.error(f"An unexpected error occurred: {e}")
#         raise

# # Example usage:
# file_path = '/kaggle/input/model-dataset-new-1/2015.csv'
# prompt = "Analyze the country and region data"
# audio_file = "/kaggle/working/"
# title_image = "/kaggle/working/"

# if not os.path.exists(file_path):
#     raise FileNotFoundError(f"Data file not found: {file_path}")
# if not os.path.exists(audio_file):
#     raise FileNotFoundError(f"Audio file not found: {audio_file}")
# if not os.path.exists(title_image):
#     raise FileNotFoundError(f"Title image not found: {title_image}")

# data_storytelling_pipeline(file_path, prompt, audio_file=audio_file)

In [36]:
# import logging
# import os
# import pandas as pd
# import time
# from moviepy.editor import ImageSequenceClip, AudioFileClip, concatenate_videoclips
# import matplotlib.pyplot as plt
# import seaborn as sns
# import numpy as np
# import io
# from PIL import Image
# from matplotlib.animation import FuncAnimation

# def generate_animated_bar_chart(data):
#     frames = []
#     fig, ax = plt.subplots(figsize=(10, 6))
#     categories = data['category'].unique()
#     bars = ax.bar(categories, np.zeros(len(categories)))
#     ax.set_ylim(0, data['value'].max())
#     ax.set_title('Moving Bar Chart')
#     ax.set_xlabel('Category')
#     ax.set_ylabel('Value')

#     def update_bar(frame):
#         for bar, category in zip(bars, categories):
#             bar.set_height(data[data['category'] == category]['value'].iloc[frame])
#         return bars

#     ani = FuncAnimation(fig, update_bar, frames=len(data), blit=True)
#     for i in range(len(data)):
#         update_bar(i)
#         buf = io.BytesIO()
#         fig.savefig(buf, format='png')
#         buf.seek(0)
#         img = Image.open(buf)
#         frames.append(np.array(img))
#     plt.close(fig)
#     return frames

# def generate_animated_correlation_matrix(data):
#     frames = []
#     corr_matrix = data.corr()
#     fig, ax = plt.subplots(figsize=(10, 6))
#     sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5, ax=ax)
#     ax.set_title('Correlation Matrix')
#     buf = io.BytesIO()
#     fig.savefig(buf, format='png')
#     buf.seek(0)
#     img = Image.open(buf)
#     frames.append(np.array(img))
#     plt.close(fig)
#     return frames

# def generate_animated_frames(data):
#     logging.info("Generating animated frames from data")
#     frames = []
    
#     # Generate animated bar chart frames
#     if 'category' in data.columns and 'value' in data.columns:
#         frames.extend(generate_animated_bar_chart(data))
    
#     # Generate animated correlation matrix frames
#     if data.select_dtypes(include=['float64', 'int64']).shape[1] > 1:
#         frames.extend(generate_animated_correlation_matrix(data))
    
#     logging.info(f"Generated {len(frames)} animated frames")
#     return frames

# def create_video_from_frames(frames, audio_file=None, video_file="module_generation_seaborn.mp4"):
#     if not frames:
#         raise ValueError("No frames to create video from.")
    
#     logging.info("Creating video from frames")
#     video_clips = [ImageSequenceClip([np.array(frame)], fps=24) for frame in frames]
    
#     video = concatenate_videoclips(video_clips, method="compose")
    
#     if audio_file and os.path.isfile(audio_file):
#         audio = AudioFileClip(audio_file)
#         video = video.set_audio(audio)
    
#     video.write_videofile(video_file, codec="libx264", fps=24)
#     logging.info(f"Video saved as {video_file}")

# def generate_infographic_video(data, insights, audio_file=None, title_image="title_screen.png"):
#     frames = generate_animated_frames(data)
#     if not frames:
#         raise ValueError("No frames generated from data.")
    
#     if os.path.exists(title_image):
#         title_image_clip = Image.open(title_image)
#         title_image_clip = title_image_clip.convert("RGBA")
#         title_image_clip = np.array(title_image_clip)
#         frames.insert(0, title_image_clip)
    
#     create_video_from_frames(frames, audio_file)
#     print("Video successfully generated!")

# def data_storytelling_pipeline(file_path, prompt, audio_file=None):
#     try:
#         start_time = time.time()
        
#         logging.info("Loading and preprocessing data...")
#         data = load_and_preprocess_data(file_path)
#         logging.debug(f"Loaded Data: {data.head()}")
        
#         logging.info("Performing EDA...")
#         eda_summary = perform_eda(data)
#         logging.debug(f"EDA Summary: {eda_summary}")
        
#         logging.info("Analyzing the user's prompt...")
#         insights = analyze_prompt_for_insights(prompt)
#         logging.debug(f"Extracted insights: {insights}")
        
#         logging.info("Creating the infographic video...")
#         generate_infographic_video(data, insights, audio_file=audio_file)
        
#         end_time = time.time()
#         logging.info(f"Pipeline completed successfully in {end_time - start_time:.2f} seconds")
    
#     except FileNotFoundError as fnf_error:
#         logging.error(f"File not found: {fnf_error}")
#         raise
#     except pd.errors.ParserError as parser_error:
#         logging.error(f"Error parsing the file: {parser_error}")
#         raise
#     except TypeError as type_error:
#         logging.error(f"Type error: {type_error}")
#         raise
#     except ValueError as value_error:
#         logging.error(f"Value error: {value_error}")
#         raise
#     except Exception as e:
#         logging.error(f"An unexpected error occurred: {e}")
#         raise

# # Example usage:
# file_path = '/kaggle/input/model-dataset-new-1/2015.csv'
# prompt = "Analyze the country and region data"
# audio_file = "/kaggle/working/"
# title_image = "/kaggle/working/"

# if not os.path.exists(file_path):
#     raise FileNotFoundError(f"Data file not found: {file_path}")
# if not os.path.exists(audio_file):
#     raise FileNotFoundError(f"Audio file not found: {audio_file}")
# if not os.path.exists(title_image):
#     raise FileNotFoundError(f"Title image not found: {title_image}")

# data_storytelling_pipeline(file_path, prompt, audio_file=audio_file)

In [37]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# import logging
# import os
# from PIL import Image
# from moviepy.editor import ImageSequenceClip, AudioFileClip, concatenate_videoclips, CompositeVideoClip, TextClip
# import numpy as np
# import io
# import pandas as pd
# import time

# def generate_default_frames(data):
#     logging.info("Generating default frames from data")
#     frames = []
    
#     # Generate a basic time series plot if a date column is present
#     date_column = None
#     for col in data.columns:
#         if 'date' in col.lower() or 'time' in col.lower():
#             date_column = col
#             break
    
#     if date_column:
#         numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns
#         if len(numeric_columns) > 0:
#             fig, ax = plt.subplots(figsize=(10, 6))
#             for col in numeric_columns:
#                 ax.plot(data[date_column], data[col], label=col)
#             ax.set_title(f"Time Series Plot for {', '.join(numeric_columns)}")
#             ax.set_xlabel('Time')
#             ax.set_ylabel('Values')
#             ax.legend()
#             buf = io.BytesIO()
#             fig.savefig(buf, format='png')
#             buf.seek(0)
#             img = Image.open(buf)
#             frames.append(img)
#             plt.close(fig)
    
#     # Generate a correlation matrix plot
#     if data.select_dtypes(include=['float64', 'int64']).shape[1] > 1:
#         corr_matrix = data.corr()
#         fig, ax = plt.subplots(figsize=(10, 6))
#         sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5, ax=ax)
#         ax.set_title('Correlation Matrix')
#         buf = io.BytesIO()
#         fig.savefig(buf, format='png')
#         buf.seek(0)
#         img = Image.open(buf)
#         frames.append(img)
#         plt.close(fig)
    
#     # Generate distribution plots for numeric columns
#     numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns
#     for col in numeric_columns:
#         fig, ax = plt.subplots(figsize=(8, 6))
#         sns.histplot(data[col], bins=10, kde=True, ax=ax)
#         ax.set_title(f"Distribution of {col}")
#         buf = io.BytesIO()
#         fig.savefig(buf, format='png')
#         buf.seek(0)
#         img = Image.open(buf)
#         frames.append(img)
#         plt.close(fig)
    
#     logging.info(f"Generated {len(frames)} default frames")
#     return frames

# def create_video_from_frames(frames, audio_file=None, video_file="frames_animations.mp4"):
#     if not frames:
#         raise ValueError("No frames to create video from.")
    
#     logging.info("Creating video from frames")
#     video_clips = []
#     for frame in frames:
#         img_clip = ImageSequenceClip([np.array(frame)], fps=1)  # 1 frame per second
#         img_clip = img_clip.set_duration(5)  # Each frame lasts 5 seconds
#         video_clips.append(img_clip)
    
#     video = concatenate_videoclips(video_clips, method="compose")
    
#     if audio_file and os.path.isfile(audio_file):
#         audio = AudioFileClip(audio_file)
#         video = video.set_audio(audio)
    
#     video.write_videofile(video_file, codec="libx264", fps=24)
#     logging.info(f"Video saved as {video_file}")

# def generate_infographic_video(data, insights, audio_file=None, title_image="title_screen.png"):
#     frames = generate_default_frames(data)
#     if not frames:
#         raise ValueError("No frames generated from data.")
    
#     if os.path.exists(title_image):
#         title_image_clip = Image.open(title_image)
#         title_image_clip = title_image_clip.convert("RGBA")
#         title_image_clip = np.array(title_image_clip)
#         frames.insert(0, title_image_clip)
    
#     create_video_from_frames(frames, audio_file)
#     print("Video successfully generated!")

# def data_storytelling_pipeline(file_path, prompt, audio_file=None):
#     try:
#         start_time = time.time()
        
#         logging.info("Loading and preprocessing data...")
#         data = load_and_preprocess_data(file_path)
#         logging.debug(f"Loaded Data: {data.head()}")
        
#         logging.info("Performing EDA...")
#         eda_summary = perform_eda(data)
#         logging.debug(f"EDA Summary: {eda_summary}")
        
#         logging.info("Analyzing the user's prompt...")
#         insights = analyze_prompt_for_insights(prompt)
#         logging.debug(f"Extracted insights: {insights}")
        
#         logging.info("Creating the infographic video...")
#         generate_infographic_video(data, insights, audio_file=audio_file)
        
#         end_time = time.time()
#         logging.info(f"Pipeline completed successfully in {end_time - start_time:.2f} seconds")
    
#     except FileNotFoundError as fnf_error:
#         logging.error(f"File not found: {fnf_error}")
#         raise
#     except pd.errors.ParserError as parser_error:
#         logging.error(f"Error parsing the file: {parser_error}")
#         raise
#     except TypeError as type_error:
#         logging.error(f"Type error: {type_error}")
#         raise
#     except ValueError as value_error:
#         logging.error(f"Value error: {value_error}")
#         raise
#     except Exception as e:
#         logging.error(f"An unexpected error occurred: {e}")
#         raise

# # Example usage:
# file_path = '/kaggle/input/model-dataset-new-1/2015.csv'
# prompt = "Analyze the country and region data"
# audio_file = "/kaggle/working/"
# title_image = "/kaggle/working/"

# if not os.path.exists(file_path):
#     raise FileNotFoundError(f"Data file not found: {file_path}")
# if not os.path.exists(audio_file):
#     raise FileNotFoundError(f"Audio file not found: {audio_file}")
# if not os.path.exists(title_image):
#     raise FileNotFoundError(f"Title image not found: {title_image}")

# data_storytelling_pipeline(file_path, prompt, audio_file=audio_file)

In [38]:
# !pip install gtts
# import matplotlib.pyplot as plt
# import seaborn as sns
# import logging
# import os
# from PIL import Image
# from moviepy.editor import ImageSequenceClip, AudioFileClip, concatenate_videoclips, CompositeVideoClip, TextClip
# import numpy as np
# import io
# import pandas as pd
# import time
# from matplotlib.animation import FuncAnimation
# from gtts import gTTS

# def generate_animated_frames(data):
#     logging.info("Generating animated frames from data")
#     frames = []
    
#     # Generate a basic time series plot if a date column is present
#     date_column = None
#     for col in data.columns:
#         if 'date' in col.lower() or 'time' in col.lower():
#             date_column = col
#             break
    
#     if date_column:
#         numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns
#         if len(numeric_columns) > 0:
#             fig, ax = plt.subplots(figsize=(10, 6))
#             lines = [ax.plot([], [], label=col)[0] for col in numeric_columns]
#             ax.set_xlim(data[date_column].min(), data[date_column].max())
#             ax.set_ylim(data[numeric_columns].min().min(), data[numeric_columns].max().max())
#             ax.set_title(f"Time Series Plot for {', '.join(numeric_columns)}")
#             ax.set_xlabel('Time')
#             ax.set_ylabel('Values')
#             ax.legend()

#             def update(frame):
#                 for line, col in zip(lines, numeric_columns):
#                     line.set_data(data[date_column][:frame], data[col][:frame])
#                 return lines

#             ani = FuncAnimation(fig, update, frames=len(data), blit=True)
#             for i in range(len(data)):
#                 update(i)
#                 buf = io.BytesIO()
#                 fig.savefig(buf, format='png')
#                 buf.seek(0)
#                 img = Image.open(buf)
#                 frames.append(np.array(img))
#             plt.close(fig)
    
#     # Generate a moving bar chart
#     if 'category' in data.columns and 'value' in data.columns:
#         fig, ax = plt.subplots(figsize=(10, 6))
#         categories = data['category'].unique()
#         bars = ax.bar(categories, np.zeros(len(categories)))
#         ax.set_ylim(0, data['value'].max())
#         ax.set_title('Moving Bar Chart')
#         ax.set_xlabel('Category')
#         ax.set_ylabel('Value')

#         def update_bar(frame):
#             for bar, category in zip(bars, categories):
#                 bar.set_height(data[data['category'] == category]['value'].iloc[frame])
#             return bars

#         ani = FuncAnimation(fig, update_bar, frames=len(data), blit=True)
#         for i in range(len(data)):
#             update_bar(i)
#             buf = io.BytesIO()
#             fig.savefig(buf, format='png')
#             buf.seek(0)
#             img = Image.open(buf)
#             frames.append(np.array(img))
#         plt.close(fig)
    
#     # Generate a correlation matrix plot
#     if data.select_dtypes(include=['float64', 'int64']).shape[1] > 1:
#         corr_matrix = data.corr()
#         fig, ax = plt.subplots(figsize=(10, 6))
#         sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5, ax=ax)
#         ax.set_title('Correlation Matrix')
#         buf = io.BytesIO()
#         fig.savefig(buf, format='png')
#         buf.seek(0)
#         img = Image.open(buf)
#         frames.append(np.array(img))
#         plt.close(fig)
    
#     # Generate distribution plots for numeric columns
#     numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns
#     for col in numeric_columns:
#         fig, ax = plt.subplots(figsize=(8, 6))
#         sns.histplot(data[col], bins=10, kde=True, ax=ax)
#         ax.set_title(f"Distribution of {col}")
#         buf = io.BytesIO()
#         fig.savefig(buf, format='png')
#         buf.seek(0)
#         img = Image.open(buf)
#         frames.append(np.array(img))
#         plt.close(fig)
    
#     logging.info(f"Generated {len(frames)} animated frames")
#     return frames

# def create_video_from_frames(frames, audio_file=None, video_file="composition_model.mp4"):
#     if not frames:
#         raise ValueError("No frames to create video from.")
    
#     logging.info("Creating video from frames")
#     video_clips = []
#     for frame in frames:
#         img_clip = ImageSequenceClip([frame], fps=1)  # 1 frame per second
#         img_clip = img_clip.set_duration(2)  # Each frame lasts 2 seconds
#         video_clips.append(img_clip)
    
#     video = concatenate_videoclips(video_clips, method="compose")
    
#     if audio_file and os.path.isfile(audio_file):
#         audio = AudioFileClip(audio_file)
#         video = video.set_audio(audio)
    
#     video.write_videofile(video_file, codec="libx264", fps=24)
#     logging.info(f"Video saved as {video_file}")

# def generate_infographic_video(data, insights, audio_file=None, title_image="title_screen.png"):
#     frames = generate_animated_frames(data)
#     if not frames:
#         raise ValueError("No frames generated from data.")
    
#     if os.path.exists(title_image):
#         title_image_clip = Image.open(title_image)
#         title_image_clip = title_image_clip.convert("RGBA")
#         title_image_clip = np.array(title_image_clip)
#         frames.insert(0, title_image_clip)
    
#     create_video_from_frames(frames, audio_file)
#     print("Video successfully generated!")

# def generate_narration(text, output_file="narration.mp3"):
#     tts = gTTS(text=text, lang='en')
#     tts.save(output_file)
#     return output_file

# def data_storytelling_pipeline(file_path, prompt, audio_file=None):
#     try:
#         start_time = time.time()
        
#         logging.info("Loading and preprocessing data...")
#         data = load_and_preprocess_data(file_path)
#         logging.debug(f"Loaded Data: {data.head()}")
        
#         logging.info("Performing EDA...")
#         eda_summary = perform_eda(data)
#         logging.debug(f"EDA Summary: {eda_summary}")
        
#         logging.info("Analyzing the user's prompt...")
#         insights = analyze_prompt_for_insights(prompt)
#         logging.debug(f"Extracted insights: {insights}")
        
#         logging.info("Generating narration...")
#         narration_text = f"Here is the analysis based on the prompt: {prompt}. {insights}"
#         narration_file = generate_narration(narration_text)
        
#         logging.info("Creating the infographic video...")
#         generate_infographic_video(data, insights, audio_file=narration_file)
        
#         end_time = time.time()
#         logging.info(f"Pipeline completed successfully in {end_time - start_time:.2f} seconds")
    
#     except FileNotFoundError as fnf_error:
#         logging.error(f"File not found: {fnf_error}")
#         raise
#     except pd.errors.ParserError as parser_error:
#         logging.error(f"Error parsing the file: {parser_error}")
#         raise
#     except TypeError as type_error:
#         logging.error(f"Type error: {type_error}")
#         raise
#     except ValueError as value_error:
#         logging.error(f"Value error: {value_error}")
#         raise
#     except Exception as e:
#         logging.error(f"An unexpected error occurred: {e}")
#         raise

# # Example usage:
# file_path = '/kaggle/input/model-dataset-new-1/2015.csv'
# prompt = "Analyze the country and region data"
# audio_file = "/kaggle/working/"
# title_image = "/kaggle/working/"

# if not os.path.exists(file_path):
#     raise FileNotFoundError(f"Data file not found: {file_path}")
# if not os.path.exists(audio_file):
#     raise FileNotFoundError(f"Audio file not found: {audio_file}")
# if not os.path.exists(title_image):
#     raise FileNotFoundError(f"Title image not found: {title_image}")

# data_storytelling_pipeline(file_path, prompt, audio_file=audio_file)

In [39]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# import logging
# import os
# from PIL import Image
# from moviepy.editor import ImageSequenceClip, AudioFileClip, concatenate_videoclips, CompositeVideoClip, TextClip
# import numpy as np
# import io
# import pandas as pd
# import time
# from matplotlib.animation import FuncAnimation
# from gtts import gTTS

# def generate_animated_gifs(data):
#     logging.info("Generating animated GIFs from data")
#     gif_files = []
    
#     # Generate a basic time series plot if a date column is present
#     date_column = None
#     for col in data.columns:
#         if 'date' in col.lower() or 'time' in col.lower():
#             date_column = col
#             break
    
#     if date_column:
#         numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns
#         if len(numeric_columns) > 0:
#             fig, ax = plt.subplots(figsize=(10, 6))
#             lines = [ax.plot([], [], label=col)[0] for col in numeric_columns]
#             ax.set_xlim(data[date_column].min(), data[date_column].max())
#             ax.set_ylim(data[numeric_columns].min().min(), data[numeric_columns].max().max())
#             ax.set_title(f"Time Series Plot for {', '.join(numeric_columns)}")
#             ax.set_xlabel('Time')
#             ax.set_ylabel('Values')
#             ax.legend()

#             def update(frame):
#                 for line, col in zip(lines, numeric_columns):
#                     line.set_data(data[date_column][:frame], data[col][:frame])
#                 return lines

#             ani = FuncAnimation(fig, update, frames=len(data), blit=True)
#             gif_file = "time_series.gif"
#             ani.save(gif_file, writer='imagemagick')
#             gif_files.append(gif_file)
#             plt.close(fig)
    
#     # Generate a moving bar chart
#     if 'category' in data.columns and 'value' in data.columns:
#         fig, ax = plt.subplots(figsize=(10, 6))
#         categories = data['category'].unique()
#         bars = ax.bar(categories, np.zeros(len(categories)))
#         ax.set_ylim(0, data['value'].max())
#         ax.set_title('Moving Bar Chart')
#         ax.set_xlabel('Category')
#         ax.set_ylabel('Value')

#         def update_bar(frame):
#             for bar, category in zip(bars, categories):
#                 bar.set_height(data[data['category'] == category]['value'].iloc[frame])
#             return bars

#         ani = FuncAnimation(fig, update_bar, frames=len(data), blit=True)
#         gif_file = "moving_bar_chart.gif"
#         ani.save(gif_file, writer='imagemagick')
#         gif_files.append(gif_file)
#         plt.close(fig)
    
#     # Generate a correlation matrix plot
#     if data.select_dtypes(include=['float64', 'int64']).shape[1] > 1:
#         corr_matrix = data.corr()
#         fig, ax = plt.subplots(figsize=(10, 6))
#         sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5, ax=ax)
#         ax.set_title('Correlation Matrix')
#         buf = io.BytesIO()
#         fig.savefig(buf, format='png')
#         buf.seek(0)
#         img = Image.open(buf)
#         img.save("correlation_matrix.png")
#         gif_files.append("correlation_matrix.png")
#         plt.close(fig)
    
#     # Generate distribution plots for numeric columns
#     numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns
#     for col in numeric_columns:
#         fig, ax = plt.subplots(figsize=(8, 6))
#         sns.histplot(data[col], bins=10, kde=True, ax=ax)
#         ax.set_title(f"Distribution of {col}")
#         buf = io.BytesIO()
#         fig.savefig(buf, format='png')
#         buf.seek(0)
#         img = Image.open(buf)
#         img.save(f"distribution_{col}.png")
#         gif_files.append(f"distribution_{col}.png")
#         plt.close(fig)
    
#     logging.info(f"Generated {len(gif_files)} animated GIFs and images")
#     return gif_files

# def create_video_from_gifs(gif_files, audio_file=None, video_file="gif_composition_video.mp4"):
#     if not gif_files:
#         raise ValueError("No GIF files to create video from.")
    
#     logging.info("Creating video from GIF files")
#     video_clips = []
#     for gif_file in gif_files:
#         if gif_file.endswith('.gif'):
#             img_clip = ImageSequenceClip(gif_file, fps=1)  # 1 frame per second
#         else:
#             img_clip = ImageSequenceClip([gif_file], fps=1)  # 1 frame per second
#         img_clip = img_clip.set_duration(5)  # Each frame lasts 5 seconds
#         video_clips.append(img_clip)
    
#     video = concatenate_videoclips(video_clips, method="compose")
    
#     if audio_file and os.path.isfile(audio_file):
#         audio = AudioFileClip(audio_file)
#         video = video.set_audio(audio)
    
#     video.write_videofile(video_file, codec="libx264", fps=24)
#     logging.info(f"Video saved as {video_file}")

# def generate_infographic_video(data, insights, audio_file=None, title_image="title_screen.png"):
#     gif_files = generate_animated_gifs(data)
#     if not gif_files:
#         raise ValueError("No GIF files generated from data.")
    
#     if os.path.exists(title_image):
#         gif_files.insert(0, title_image)
    
#     create_video_from_gifs(gif_files, audio_file)
#     print("Video successfully generated!")

# def generate_narration(text, output_file="narration.mp3"):
#     tts = gTTS(text=text, lang='en')
#     tts.save(output_file)
#     return output_file

# def data_storytelling_pipeline(file_path, prompt, audio_file=None):
#     try:
#         start_time = time.time()
        
#         logging.info("Loading and preprocessing data...")
#         data = load_and_preprocess_data(file_path)
#         logging.debug(f"Loaded Data: {data.head()}")
        
#         logging.info("Performing EDA...")
#         eda_summary = perform_eda(data)
#         logging.debug(f"EDA Summary: {eda_summary}")
        
#         logging.info("Analyzing the user's prompt...")
#         insights = analyze_prompt_for_insights(prompt)
#         logging.debug(f"Extracted insights: {insights}")
        
#         logging.info("Generating narration...")
#         narration_text = f"Here is the analysis based on the prompt: {prompt}. {insights}"
#         narration_file = generate_narration(narration_text)
        
#         logging.info("Creating the infographic video...")
#         generate_infographic_video(data, insights, audio_file=narration_file)
        
#         end_time = time.time()
#         logging.info(f"Pipeline completed successfully in {end_time - start_time:.2f} seconds")
    
#     except FileNotFoundError as fnf_error:
#         logging.error(f"File not found: {fnf_error}")
#         raise
#     except pd.errors.ParserError as parser_error:
#         logging.error(f"Error parsing the file: {parser_error}")
#         raise
#     except TypeError as type_error:
#         logging.error(f"Type error: {type_error}")
#         raise
#     except ValueError as value_error:
#         logging.error(f"Value error: {value_error}")
#         raise
#     except Exception as e:
#         logging.error(f"An unexpected error occurred: {e}")
#         raise

# # Example usage:
# file_path = '/kaggle/input/model-dataset-new-1/2015.csv'
# prompt = "Analyze the country and region data"
# audio_file = "/kaggle/working/"
# title_image = "/kaggle/working/"

# if not os.path.exists(file_path):
#     raise FileNotFoundError(f"Data file not found: {file_path}")
# if not os.path.exists(audio_file):
#     raise FileNotFoundError(f"Audio file not found: {audio_file}")
# if not os.path.exists(title_image):
#     raise FileNotFoundError(f"Title image not found: {title_image}")

# data_storytelling_pipeline(file_path, prompt, audio_file=audio_file)

In [40]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# import logging
# import os
# from PIL import Image
# from moviepy.editor import ImageSequenceClip, AudioFileClip, concatenate_videoclips, CompositeVideoClip, TextClip
# import numpy as np
# import io
# import pandas as pd
# import time
# from matplotlib.animation import FuncAnimation
# from gtts import gTTS

# def generate_animated_frames(data):
#     logging.info("Generating animated frames from data")
#     frames = []
    
#     # Generate a basic time series plot if a date column is present
#     date_column = None
#     for col in data.columns:
#         if 'date' in col.lower() or 'time' in col.lower():
#             date_column = col
#             break
    
#     if date_column:
#         numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns
#         if len(numeric_columns) > 0:
#             fig, ax = plt.subplots(figsize=(10, 6))
#             lines = [ax.plot([], [], label=col)[0] for col in numeric_columns]
#             ax.set_xlim(data[date_column].min(), data[date_column].max())
#             ax.set_ylim(data[numeric_columns].min().min(), data[numeric_columns].max().max())
#             ax.set_title(f"Time Series Plot for {', '.join(numeric_columns)}")
#             ax.set_xlabel('Time')
#             ax.set_ylabel('Values')
#             ax.legend()

#             def update(frame):
#                 for line, col in zip(lines, numeric_columns):
#                     line.set_data(data[date_column][:frame], data[col][:frame])
#                 return lines

#             ani = FuncAnimation(fig, update, frames=len(data), blit=True)
#             for i in range(len(data)):
#                 update(i)
#                 buf = io.BytesIO()
#                 fig.savefig(buf, format='png')
#                 buf.seek(0)
#                 img = Image.open(buf)
#                 frames.append(np.array(img))
#             plt.close(fig)
    
#     # Generate an animated bar chart
#     if 'category' in data.columns and 'value' in data.columns:
#         fig, ax = plt.subplots(figsize=(10, 6))
#         categories = data['category'].unique()
#         bars = ax.bar(categories, np.zeros(len(categories)))
#         ax.set_ylim(0, data['value'].max())
#         ax.set_title('Animated Bar Chart')
#         ax.set_xlabel('Category')
#         ax.set_ylabel('Value')

#         def update_bar(frame):
#             for bar, category in zip(bars, categories):
#                 bar.set_height(data[data['category'] == category]['value'].iloc[frame])
#             return bars

#         ani = FuncAnimation(fig, update_bar, frames=len(data), blit=True)
#         for i in range(len(data)):
#             update_bar(i)
#             buf = io.BytesIO()
#             fig.savefig(buf, format='png')
#             buf.seek(0)
#             img = Image.open(buf)
#             frames.append(np.array(img))
#         plt.close(fig)
    
#     # Generate an animated pie chart
#     if 'category' in data.columns and 'value' in data.columns:
#         fig, ax = plt.subplots(figsize=(10, 6))
#         def update_pie(frame):
#             ax.clear()
#             ax.pie(data['value'][:frame], labels=data['category'][:frame], autopct='%1.1f%%')
#             ax.set_title('Animated Pie Chart')

#         ani = FuncAnimation(fig, update_pie, frames=len(data), blit=True)
#         for i in range(len(data)):
#             update_pie(i)
#             buf = io.BytesIO()
#             fig.savefig(buf, format='png')
#             buf.seek(0)
#             img = Image.open(buf)
#             frames.append(np.array(img))
#         plt.close(fig)
    
#     # Generate a correlation matrix plot
#     if data.select_dtypes(include=['float64', 'int64']).shape[1] > 1:
#         corr_matrix = data.corr()
#         fig, ax = plt.subplots(figsize=(10, 6))
#         sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5, ax=ax)
#         ax.set_title('Correlation Matrix')
#         buf = io.BytesIO()
#         fig.savefig(buf, format='png')
#         buf.seek(0)
#         img = Image.open(buf)
#         frames.append(np.array(img))
#         plt.close(fig)
    
#     # Generate distribution plots for numeric columns
#     numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns
#     for col in numeric_columns:
#         fig, ax = plt.subplots(figsize=(8, 6))
#         sns.histplot(data[col], bins=10, kde=True, ax=ax)
#         ax.set_title(f"Distribution of {col}")
#         buf = io.BytesIO()
#         fig.savefig(buf, format='png')
#         buf.seek(0)
#         img = Image.open(buf)
#         frames.append(np.array(img))
#         plt.close(fig)
    
#     logging.info(f"Generated {len(frames)} animated frames")
#     return frames

# def create_video_from_frames(frames, audio_file=None, video_file="gif_new_composition.mp4"):
#     if not frames:
#         raise ValueError("No frames to create video from.")
    
#     logging.info("Creating video from frames")
#     video_clips = []
#     for frame in frames:
#         img_clip = ImageSequenceClip([frame], fps=1)  # 1 frame per second
#         img_clip = img_clip.set_duration(2)  # Each frame lasts 2 seconds
#         video_clips.append(img_clip)
    
#     video = concatenate_videoclips(video_clips, method="compose")
    
#     if audio_file and os.path.isfile(audio_file):
#         audio = AudioFileClip(audio_file)
#         video = video.set_audio(audio)
    
#     video.write_videofile(video_file, codec="libx264", fps=24)
#     logging.info(f"Video saved as {video_file}")

# def generate_infographic_video(data, insights, audio_file=None, title_image="title_screen.png"):
#     frames = generate_animated_frames(data)
#     if not frames:
#         raise ValueError("No frames generated from data.")
    
#     if os.path.exists(title_image):
#         title_image_clip = Image.open(title_image)
#         title_image_clip = title_image_clip.convert("RGBA")
#         title_image_clip = np.array(title_image_clip)
#         frames.insert(0, title_image_clip)
    
#     create_video_from_frames(frames, audio_file)
#     print("Video successfully generated!")

# def generate_narration(text, output_file="narration.mp3"):
#     tts = gTTS(text=text, lang='en')
#     tts.save(output_file)
#     return output_file

# def data_storytelling_pipeline(file_path, prompt, audio_file=None):
#     try:
#         start_time = time.time()
        
#         logging.info("Loading and preprocessing data...")
#         data = load_and_preprocess_data(file_path)
#         logging.debug(f"Loaded Data: {data.head()}")
        
#         logging.info("Performing EDA...")
#         eda_summary = perform_eda(data)
#         logging.debug(f"EDA Summary: {eda_summary}")
        
#         logging.info("Analyzing the user's prompt...")
#         insights = analyze_prompt_for_insights(prompt)
#         logging.debug(f"Extracted insights: {insights}")
        
#         logging.info("Generating narration...")
#         narration_text = f"Here is the analysis based on the prompt: {prompt}. {insights}"
#         narration_file = generate_narration(narration_text)
        
#         logging.info("Creating the infographic video...")
#         generate_infographic_video(data, insights, audio_file=narration_file)
        
#         end_time = time.time()
#         logging.info(f"Pipeline completed successfully in {end_time - start_time:.2f} seconds")
    
#     except FileNotFoundError as fnf_error:
#         logging.error(f"File not found: {fnf_error}")
#         raise
#     except pd.errors.ParserError as parser_error:
#         logging.error(f"Error parsing the file: {parser_error}")
#         raise
#     except TypeError as type_error:
#         logging.error(f"Type error: {type_error}")
#         raise
#     except ValueError as value_error:
#         logging.error(f"Value error: {value_error}")
#         raise
#     except Exception as e:
#         logging.error(f"An unexpected error occurred: {e}")
#         raise

# # Example usage:
# file_path = '/kaggle/input/model-dataset-new-1/2017.csv'
# prompt = "compare the family and generaosity in an interactive video format"
# audio_file = "/kaggle/working/"
# title_image = "/kaggle/working/"

# if not os.path.exists(file_path):
#     raise FileNotFoundError(f"Data file not found: {file_path}")
# if not os.path.exists(audio_file):
#     raise FileNotFoundError(f"Audio file not found: {audio_file}")
# if not os.path.exists(title_image):
#     raise FileNotFoundError(f"Title image not found: {title_image}")

# data_storytelling_pipeline(file_path, prompt, audio_file=audio_file)

In [41]:
# import plotly.express as px
# import pandas as pd
# import logging
# import os
# import time
# from moviepy.editor import ImageSequenceClip, concatenate_videoclips, AudioFileClip, VideoFileClip
# from gtts import gTTS
# from transformers import BartTokenizer, BartForConditionalGeneration
# import spacy
# import re
# from sklearn.preprocessing import StandardScaler

# # Setting up logging
# logging.basicConfig(level=logging.INFO)

# # Load spaCy model for NER
# nlp = spacy.load("en_core_web_sm")
# import plotly.express as px
# import pandas as pd
# import logging
# import os
# import time
# from moviepy.editor import ImageSequenceClip, concatenate_videoclips, AudioFileClip, VideoFileClip
# from gtts import gTTS
# from transformers import BartTokenizer, BartForConditionalGeneration
# import spacy
# import re
# from sklearn.preprocessing import StandardScaler

# # Setting up logging
# logging.basicConfig(level=logging.INFO)

# # Load spaCy model for NER
# nlp = spacy.load("en_core_web_sm")

# def load_and_preprocess_data(file_path):
#     try:
#         logging.info(f"Loading data from {file_path}")
#         print(f"Loading data from {file_path}")
#         # Load data
#         if file_path.endswith('.csv'):
#             data = pd.read_csv(file_path)
#         elif file_path.endswith('.xlsx'):
#             data = pd.read_excel(file_path)
#         elif file_path.endswith('.txt'):
#             data = pd.read_csv(file_path, delimiter='\t')
#         else:
#             raise ValueError("Unsupported file format.")
        
#         logging.info("Data loaded successfully")
#         print("Data loaded successfully")
        
#         # Detect and convert data types
#         for col in data.columns:
#             try:
#                 data[col] = pd.to_numeric(data[col], errors='coerce')
#             except ValueError:
#                 pass
        
#         # Handle missing values
#         data.fillna(data.mean(), inplace=True)
        
#         # Handle categorical data
#         categorical_cols = data.select_dtypes(include=['object']).columns
#         data = pd.get_dummies(data, columns=categorical_cols, drop_first=True)
        
#         # Normalize numeric data
#         numeric_cols = data.select_dtypes(include=['float64', 'int64']).columns
#         scaler = StandardScaler()
#         data[numeric_cols] = scaler.fit_transform(data[numeric_cols])
        
#         # Add default columns if missing
#         if 'date' not in data.columns:
#             data['date'] = pd.date_range(start='1/1/2020', periods=len(data), freq='D')
#         if 'value' not in data.columns:
#             data['value'] = 1  # Default value
#         if 'category' not in data.columns:
#             data['category'] = 'default_category'
        
#         logging.info("Data preprocessing completed")
#         print("Data preprocessing completed")
#         return data
#     except Exception as e:
#         logging.error(f"Error loading and preprocessing data: {e}")
#         print(f"Error loading and preprocessing data: {e}")
#         raise

# # The rest of the code remains the same

# def infer_columns(data):
#     try:
#         column_mapping = {}
#         possible_columns = {
#             'category': ['category', 'type', 'class', 'label', 'country', 'region'],
#             'value': ['value', 'amount', 'score', 'total', 'family', 'generosity'],
#             'date': ['date', 'time', 'year', 'month', 'day'],
#             'x': ['x', 'longitude', 'lat', 'latitude'],
#             'y': ['y', 'latitude', 'long', 'longitude']
#         }
        
#         logging.info(f"Data columns: {data.columns.tolist()}")
#         print(f"Data columns: {data.columns.tolist()}")
#         for key, patterns in possible_columns.items():
#             for pattern in patterns:
#                 potential_cols = [col for col in data.columns if re.search(pattern, col, re.IGNORECASE)]
#                 if potential_cols:
#                     column_mapping[key] = potential_cols[0]
#                     break
        
#         # If any required column is missing, attempt to infer it
#         if 'date' not in column_mapping:
#             data['date'] = pd.date_range(start='1/1/2020', periods=len(data), freq='D')
#             column_mapping['date'] = 'date'
        
#         if 'value' not in column_mapping:
#             numeric_cols = data.select_dtypes(include=['float64', 'int64']).columns
#             if len(numeric_cols) > 0:
#                 column_mapping['value'] = numeric_cols[0]
#             else:
#                 raise ValueError("Cannot infer 'value' column.")
        
#         if 'category' not in column_mapping:
#             categorical_cols = data.select_dtypes(include=['object']).columns
#             if len(categorical_cols) > 0:
#                 column_mapping['category'] = categorical_cols[0]
#             else:
#                 non_numeric_cols = data.select_dtypes(exclude=['float64', 'int64']).columns
#                 if len(non_numeric_cols) > 0:
#                     column_mapping['category'] = non_numeric_cols[0]
#                 else:
#                     raise ValueError("Cannot infer 'category' column.")
        
#         return column_mapping
#     except Exception as e:
#         logging.error(f"Error inferring columns: {e}")
#         print(f"Error inferring columns: {e}")
#         raise

# def generate_frames(data, chart_type, column_mapping, output_dir="frames"):
#     try:
#         if not os.path.exists(output_dir):
#             os.makedirs(output_dir)
        
#         frames = []
#         for date in data[column_mapping['date']].unique():
#             filtered_data = data[data[column_mapping['date']] == date]
#             if chart_type == 'bar':
#                 fig = px.bar(filtered_data, x=column_mapping['category'], y=column_mapping['value'], title=f"Bar Chart - {date}")
#             elif chart_type == 'pie':
#                 fig = px.pie(filtered_data, values=column_mapping['value'], names=column_mapping['category'], title=f"Pie Chart - {date}")
#             elif chart_type == 'scatter':
#                 fig = px.scatter(filtered_data, x=column_mapping['x'], y=column_mapping['y'], size=column_mapping['value'], color=column_mapping['category'], title=f"Scatter Plot - {date}")
#             else:
#                 raise ValueError("Invalid chart type")
            
#             frame_path = os.path.join(output_dir, f"{chart_type}_{date}.png")
#             fig.write_image(frame_path)
#             frames.append(frame_path)
        
#         return frames
#     except Exception as e:
#         logging.error(f"Error generating frames: {e}")
#         print(f"Error generating frames: {e}")
#         raise

# def generate_video_from_frames(frames, output_file="animation.mp4", fps=1):
#     try:
#         clip = ImageSequenceClip(frames, fps=fps)
#         clip.write_videofile(output_file, codec="libx264")
#         return output_file
#     except Exception as e:
#         logging.error(f"Error generating video from frames: {e}")
#         print(f"Error generating video from frames: {e}")
#         raise

# def generate_infographic_video(data, insights, audio_file=None, title_image="title_screen.png"):
#     try:
#         logging.info("Checking data columns for required visualizations")
#         print("Checking data columns for required visualizations")
#         column_mapping = infer_columns(data)
#         logging.debug(f"Column mapping: {column_mapping}")
#         print(f"Column mapping: {column_mapping}")
#         video_files = generate_videos_if_needed(data, column_mapping)
        
#         if not video_files:
#             logging.error("No video files generated from data. Check if the data has the required columns.")
#             print("No video files generated from data. Check if the data has the required columns.")
#             raise ValueError("No video files generated from data.")
        
#         logging.info("Combining video files into a single video")
#         print("Combining video files into a single video")
#         video_clips = [VideoFileClip(video_file) for video_file in video_files]
        
#         if os.path.exists(title_image):
#             title_clip = VideoFileClip(title_image).set_duration(5)
#             video_clips.insert(0, title_clip)
        
#         final_video = concatenate_videoclips(video_clips, method="compose")
        
#         if audio_file and os.path.isfile(audio_file):
#             audio = AudioFileClip(audio_file)
#             final_video = final_video.set_audio(audio)
        
#         final_video.write_videofile("attemot_video.mp4", codec="libx264", fps=24)
#         logging.info("Final video saved as final_video.mp4")
#         print("Final video saved as final_video.mp4")
#     except Exception as e:
#         logging.error(f"Error generating infographic video: {e}")
# def generate_videos_if_needed(data, column_mapping):
#         raise

# def generate_videos_if_needed(data):
#     try:
#         video_files = []
#         column_mapping = infer_columns(data)
#         required_columns = {'category', 'value', 'date'}
#         if not required_columns.issubset(column_mapping.keys()):
#             logging.error(f"Missing required columns: {required_columns - column_mapping.keys()}")
#             print(f"Missing required columns: {required_columns - column_mapping.keys()}")
#             raise ValueError(f"Missing required columns: {required_columns - column_mapping.keys()}")

#         if required_columns.issubset(column_mapping.keys()):
#             logging.info("Data contains required columns for bar and pie charts")
#             print("Data contains required columns for bar and pie charts")
#             try:
#                 bar_frames = generate_frames(data, 'bar', column_mapping)
#                 video_files.append(generate_video_from_frames(bar_frames, "animated_bar_chart.mp4"))
#             except Exception as e:
#                 logging.error(f"Failed to generate animated bar chart: {e}")
#                 print(f"Failed to generate animated bar chart: {e}")
#             try:
#                 pie_frames = generate_frames(data, 'pie', column_mapping)
#                 video_files.append(generate_video_from_frames(pie_frames, "animated_pie_chart.mp4"))
#             except Exception as e:
#                 logging.error(f"Failed to generate animated pie chart: {e}")
#                 print(f"Failed to generate animated pie chart: {e}")
#         if {'x', 'y', 'value', 'date'}.issubset(column_mapping.keys()):
#             logging.info("Data contains required columns for scatter plot")
#             print("Data contains required columns for scatter plot")
#             try:
#                 scatter_frames = generate_frames(data, 'scatter', column_mapping)
#                 video_files.append(generate_video_from_frames(scatter_frames, "animated_scatter_plot.mp4"))
#             except Exception as e:
#                 logging.error(f"Failed to generate animated scatter plot: {e}")
#                 print(f"Failed to generate animated scatter plot: {e}")
#         return video_files
#     except Exception as e:
#         logging.error(f"Error generating videos if needed: {e}")
#         print(f"Error generating videos if needed: {e}")
#         raise

# def generate_narration(text, output_file="narration.mp3"):
#     try:
#         tts = gTTS(text=text, lang='en')
#         tts.save(output_file)
#         return output_file
#     except Exception as e:
#         logging.error(f"Error generating narration: {e}")
#         print(f"Error generating narration: {e}")
#         raise

# def perform_eda(data):
#     try:
#         eda_summary = {
#             "shape": data.shape,
#             "columns": data.columns.tolist(),
#             "dtypes": data.dtypes.tolist(),
#             "null_counts": data.isnull().sum().tolist(),
#             "describe": data.describe().to_dict()
#         }
#         return eda_summary
#     except Exception as e:
#         logging.error(f"Error performing EDA: {e}")
#         print(f"Error performing EDA: {e}")
#         raise

# def analyze_prompt_for_insights(prompt, model_name="facebook/bart-large"):
#     try:
#         logging.info("Analyzing prompt for insights")
#         print("Analyzing prompt for insights")
#         tokenizer = BartTokenizer.from_pretrained(model_name)
#         model = BartForConditionalGeneration.from_pretrained(model_name)
        
#         inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
#         input_ids = inputs.input_ids
#         attention_mask = inputs.attention_mask
        
#         outputs = model.generate(
#             input_ids,
#             attention_mask=attention_mask,
#             max_length=200,
#             num_return_sequences=1,
#             temperature=0.7,
#             top_p=0.9,
#             top_k=50,
#             no_repeat_ngram_size=2,
#             pad_token_id=tokenizer.eos_token_id
#         )
        
#         generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
#         insights = extract_insights_from_text(generated_text)
        
#         insights_list = [key for key, value in insights.items() if value]
#         if not insights_list:
#             logging.warning("No insights could be extracted from the provided prompt. Using default insights.")
#             print("No insights could be extracted from the provided prompt. Using default insights.")
#             insights_list = ["trend", "comparison"]
        
#         logging.info(f"Insights extracted: {insights_list}")
#         print(f"Insights extracted: {insights_list}")
#         return insights_list
#     except Exception as e:
#         logging.error(f"Error in analyzing prompt for insights: {e}")
#         print(f"Error in analyzing prompt for insights: {e}")
#         return ["trend", "comparison"]

# def data_storytelling_pipeline(file_path, prompt, audio_file=None):
#     try:
#         start_time = time.time()
        
#         logging.info("Loading and preprocessing data...")
#         print("Loading and preprocessing data...")
#         data = load_and_preprocess_data(file_path)
#         logging.debug(f"Loaded Data: {data.head()}")
#         print(f"Loaded Data: {data.head()}")
        
#         logging.info("Performing EDA...")
#         print("Performing EDA...")
#         eda_summary = perform_eda(data)
#         logging.debug(f"EDA Summary: {eda_summary}")
#         print(f"EDA Summary: {eda_summary}")
        
#         logging.info("Analyzing the user's prompt...")
#         print("Analyzing the user's prompt...")
#         insights = analyze_prompt_for_insights(prompt)
#         logging.debug(f"Extracted insights: {insights}")
#         print(f"Extracted insights: {insights}")
        
#         logging.info("Generating narration...")
#         print("Generating narration...")
#         narration_text = f"Here is the analysis based on the prompt: {prompt}. {insights}"
#         narration_file = generate_narration(narration_text)
        
#         logging.info("Creating the infographic video...")
#         print("Creating the infographic video...")
#         generate_infographic_video(data, insights, audio_file=narration_file)
        
#         end_time = time.time()
#         logging.info(f"Pipeline completed successfully in {end_time - start_time:.2f} seconds")
#         print(f"Pipeline completed successfully in {end_time - start_time:.2f} seconds")
    
#     except FileNotFoundError as fnf_error:
#         logging.error(f"File not found: {fnf_error}")
#         print(f"File not found: {fnf_error}")
#         raise
#     except pd.errors.ParserError as parser_error:
#         logging.error(f"Error parsing the file: {parser_error}")
#         print(f"Error parsing the file: {parser_error}")
#         raise
#     except TypeError as type_error:
#         logging.error(f"Type error: {type_error}")
#         print(f"Type error: {type_error}")
#         raise
#     except ValueError as value_error:
#         logging.error(f"Value error: {value_error}")
#         print(f"Value error: {value_error}")
#         raise
#     except Exception as e:
#         logging.error(f"An unexpected error occurred: {e}")
#         print(f"An unexpected error occurred: {e}")
#         raise

# # Example usage:
# file_path = '/kaggle/input/model-dataset-new-1/2017.csv'
# prompt = "analyse the family and generaosity columns data in the video format"
# audio_file = "/kaggle/working/"
# title_image = "/kaggle/working/"

# if not os.path.exists(file_path):
#     raise FileNotFoundError(f"Data file not found: {file_path}")
# if not os.path.exists(audio_file):
#     raise FileNotFoundError(f"Audio file not found: {audio_file}")
# if not os.path.exists(title_image):
#     raise FileNotFoundError(f"Title image not found: {title_image}")

# try:
#     data_storytelling_pipeline(file_path, prompt, audio_file=audio_file)
# except ValueError as e:
#     if "Missing required columns" in str(e):
#         logging.info("Adding missing columns to the data")
#         print("Adding missing columns to the data")
#         data = load_and_preprocess_data(file_path)
#         column_mapping = infer_columns(data)
        
#         if 'date' not in column_mapping:
#             data['date'] = pd.date_range(start='1/1/2020', periods=len(data), freq='D')
        
#         if 'value' not in column_mapping:
#             data['value'] = data.select_dtypes(include=['float64', 'int64']).iloc[:, 0]
        
#         if 'category' not in column_mapping:
#             data['category'] = 'default_category'
        
#         data_storytelling_pipeline(file_path, prompt, audio_file=audio_file)
#     else:
#         raise

In [42]:
# will work tommorow